In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_colwidth", 200)

/Users/lisakim/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Step 1: Clean median income data

In [2]:
import os
os.chdir("MedianIncomeData")

In [3]:
!ls

ACSDT5Y2019.B19013_data_with_overlays_2021-03-28T171056 2.xlsx
ACSDT5Y2019.B19013_data_with_overlays_2021-03-28T171056.csv
ACSDT5Y2019.B19013_metadata_2021-03-28T171056.csv
ACSDT5Y2019.B19013_table_title_2021-03-28T171056.txt
Burglary2021.csv
Burglary_NarrowDown.csv
GrandLarceny2021.csv
NYC_IncomeLevels.xlsx
NYC_MajorFelony_CrimeStats_New4.xlsx
OLD_df_MI_P1.csv
PCT_GrandLarceny.csv
ShootingsPercentChange.csv
VehicleTheft2021.csv
ZipCode_Income.xlsx
ZipCode_Income2.xlsx
ZipCode_Income3.csv
carTheft_Pct.csv
change_19_20.csv
change_20_21.csv
df_MI.csv
df_MI_P1.xlsx
df_MI_Precincts.xlsx
df_MI_Precincts_New.xlsx
dfinal_new
drop_19_20.csv
drop_20_21.csv
lowerIncome.csv
lowerMiddleIncome.csv
rise_19_20.csv
rise_20_21.csv
upperIncome.csv
upperMiddleIncome.csv
~$ACSDT5Y2019.B19013_data_with_overlays_2021-03-28T171056 2.xlsx


In [4]:
df_MI = pd.read_csv("ACSDT5Y2019.B19013_data_with_overlays_2021-03-28T171056.csv")

In [5]:
df_MI.head(10)

,GEO_ID,NAME,B19013_001E,B19013_001M
0,id,Geographic Area Name,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars),Margin of Error!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)
1,8600000US06390,ZCTA5 06390,61125,8799
2,8600000US10001,ZCTA5 10001,92840,8971
3,8600000US10002,ZCTA5 10002,36982,2384
4,8600000US10003,ZCTA5 10003,118161,10235
5,8600000US10004,ZCTA5 10004,190223,49426
6,8600000US10005,ZCTA5 10005,189702,15832
7,8600000US10006,ZCTA5 10006,179044,16237
8,8600000US10007,ZCTA5 10007,224063,45620
9,8600000US10009,ZCTA5 10009,63717,4878


In [6]:
df_MI.columns = df_MI.iloc[0]
df_MI = df_MI[1:]

In [7]:
df_MI.head(2)

,id,Geographic Area Name,Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars),Margin of Error!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)
1,8600000US06390,ZCTA5 06390,61125,8799
2,8600000US10001,ZCTA5 10001,92840,8971


In [8]:
df_MI = df_MI.rename(columns={"Estimate!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)": "Median household income2019", "Margin of Error!!Median household income in the past 12 months (in 2019 inflation-adjusted dollars)": "Margin of Error"})

In [9]:
df_MI = df_MI.rename(columns={"Geographic Area Name": "Zip Code"})

In [10]:
df_MI['Zip Code'] = df_MI['Zip Code'].str.replace("ZCTA5 ", "")

In [11]:
df_MI.head(2)

,id,Zip Code,Median household income2019,Margin of Error
1,8600000US06390,06390,61125,8799
2,8600000US10001,10001,92840,8971


In [12]:
# https://www1.nyc.gov/assets/doh/downloads/pdf/ah/zipcodetable.pdf

In [13]:
df_MI['Precinct'] = df_MI['Zip Code'].replace("10463", "50", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10471", "50", regex=False)

df_MI['NeighName'] = df_MI['Zip Code'].str.replace("10463", "Kingsbridge - Riverdale", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10471", "Kingsbridge - Riverdale", regex=False)

In [14]:
df_MI['Precinct'] = df_MI['Precinct'].replace("10466", "49, 45", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10469", "49, 45", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10470", "49, 45", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10475", "49, 45", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10466", "Northeast Bronx", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10469", "Northeast Bronx", regex=False)
df_MI['NeighName'] =df_MI['NeighName'].str.replace("10470", "Northeast Bronx", regex=False)
df_MI['NeighName'] =df_MI['NeighName'].str.replace("10475", "Northeast Bronx", regex=False)

In [15]:
df_MI['Precinct'] = df_MI['Precinct'].replace("10458", "52, 46", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10467", "52, 46", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10468", "52, 46", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10458", "Fordham - Bronx Park", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10467", "Fordham - Bronx Park", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10468", "Fordham - Bronx Park", regex=False)

In [16]:
df_MI['Precinct'] = df_MI['Precinct'].replace("10461", "45", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10462", "45", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10464", "45", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10465", "45", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10472", "45", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10473", "45", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10461", "Southeast Bronx", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10462", "Southeast Bronx", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10464", "Southeast Bronx", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10465", "Southeast Bronx", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10472", "Southeast Bronx", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10473", "Southeast Bronx", regex=False)

In [17]:
df_MI['Precinct'] = df_MI['Precinct'].replace("10453", "48, 42", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10457", "48, 42", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10460", "48, 42", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10453", "Central Bronx", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10457", "Central Bronx", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10460", "Central Bronx", regex=False)


In [18]:
df_MI['Precinct'] = df_MI['Precinct'].replace("10451", "44", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10452", "44", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10456", "44", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10451", "High Bridge - Morrisania", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10452", "High Bridge - Morrisania", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10456", "High Bridge - Morrisania", regex=False)


In [19]:
df_MI['Precinct'] = df_MI['Precinct'].replace("10454", "41, 40", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10455", "41, 40", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10459", "41, 40", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10474", "41, 40", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10454", "Hunts Point - Mott Haven", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10455", "Hunts Point - Mott Haven", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10459", "Hunts Point - Mott Haven", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10474", "Hunts Point - Mott Haven", regex=False)


In [20]:
df_MI['Precinct'] = df_MI['Precinct'].replace("11211", "94", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11222", "94", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11211", "Greenpoint", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11222", "Greenpoint", regex=False)


In [21]:
df_MI['Precinct'] = df_MI['Precinct'].replace("11201", "78", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11205", "78", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11215", "78", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11217", "78", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11231", "78", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11201", "Northwest Brooklyn", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11205", "Northwest Brooklyn", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11215", "Northwest Brooklyn", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11217", "Northwest Brooklyn", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11231", "Northwest Brooklyn", regex=False)


In [22]:
# Bedford Stuyvesant - Crown Heights 77 81 71
# 11213, 11212, 11216, 11233, 11238

df_MI['Precinct'] = df_MI['Precinct'].replace("11213", "77, 81, 71", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11212", "77, 81, 71", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11216", "77, 81, 71", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11233", "77, 81, 71", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11238", "77, 81, 71", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11213", "Central Brooklyn", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11212", "Central Brooklyn", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11216", "Central Brooklyn", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11233", "Central Brooklyn", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11238", "Central Brooklyn", regex=False)


In [23]:
# East New York 75 11207, 11208
df_MI['Precinct'] = df_MI['Precinct'].replace("11207", "75", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11208", "75", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11207", "East New York", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11208", "East New York", regex=False)


In [24]:
# Sunset Park 72 68 
# 11220, 11232
df_MI['Precinct'] = df_MI['Precinct'].replace("11220", "72, 68", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11232", "72, 76", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11220", "Sunset Park", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11232", "Sunset Park", regex=False)

In [25]:
# Borough Park 66 
# 11204, 11218, 11219, 11230

df_MI['Precinct'] = df_MI['Precinct'].replace("11204", "66", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11218", "66", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11219", "66", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11230", "66", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11204", "Borough Park", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11218", "Borough Park", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11219", "Borough Park", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11230", "Borough Park", regex=False)


In [26]:
# East Flatbush - Flatbush 67 70
# 11203, 11210, 11225, 11226

df_MI['Precinct'] = df_MI['Precinct'].replace("11203", "67, 70", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11210", "67, 70", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11225", "67, 70", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11226", "67, 70", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11203", "East Flatbush - Flatbush", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11210", "East Flatbush - Flatbush", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11225", "East Flatbush - Flatbush", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11226", "East Flatbush - Flatbush", regex=False)


In [27]:
# Canarsie - Flatlands 69 63
# 11234, 11236, 11239
df_MI['Precinct'] = df_MI['Precinct'].replace("11234", "69, 63", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11236", "69, 63", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11239", "69, 63", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11234", "Canarsie - Flatlands", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11236", "Canarsie - Flatlands", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11239", "Canarsie - Flatlands", regex=False)


In [28]:
# Bensonhurst - Bay Ridge 68 62
# 11209, 11214, 11228

df_MI['Precinct'] = df_MI['Precinct'].replace("11209", "68, 62", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11214", "68, 62", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11228", "68, 62", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11209", "Southwest Brooklyn", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11214", "Southwest Brooklyn", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11228", "Southwest Brooklyn", regex=False)


In [29]:
# Coney Island - Sheepshead Bay 61
# 11223, 11224, 11229, 11235

df_MI['Precinct'] = df_MI['Precinct'].replace("11223", "61", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11224", "61", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11229", "61", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11235", "61", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11223", "Southern Brooklyn", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11224", "Southern Brooklyn", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11229", "Southern Brooklyn", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11235", "Southern Brooklyn", regex=False)

In [30]:
# Williamsburg - Bushwick 83 
# 11206, 11221, 11237

df_MI['Precinct'] = df_MI['Precinct'].replace("11206", "83", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11221", "83", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11237", "83", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11206", "Williamsburg - Bushwick", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11221", "Williamsburg - Bushwick", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11237", "Williamsburg - Bushwick", regex=False)


In [31]:
# Washington Heights - Inwood 33 34
# 10031, 10032, 10033, 10034, 10040

df_MI['Precinct'] = df_MI['Precinct'].replace("10031", "33, 34", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10032", "33, 34", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10033", "33, 34", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10034", "33, 34", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10040", "33, 34", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10031", "Washington Heights - Inwood", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10032", "Washington Heights - Inwood", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10033", "Washington Heights - Inwood", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10034", "Washington Heights - Inwood", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10040", "Washington Heights - Inwood", regex=False)

In [32]:
# Central Harlem - Morningside Heights 26 28
# 10026, 10027, 10030, 10037, 10039

df_MI['Precinct'] = df_MI['Precinct'].replace("10026", "26, 28", regex=False)    
df_MI['Precinct'] = df_MI['Precinct'].replace("10027", "26, 28", regex=False) 
df_MI['Precinct'] = df_MI['Precinct'].replace("10030", "26, 28", regex=False) 
df_MI['Precinct'] =df_MI['Precinct'].replace("10037", "26, 28", regex=False) 
df_MI['Precinct'] = df_MI['Precinct'].replace("10039", "26, 28", regex=False) 

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10026", "Central Harlem - Morningside Heights", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10027", "Central Harlem - Morningside Heights", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10030", "Central Harlem - Morningside Heights", regex=False)
df_MI['NeighName'] =df_MI['NeighName'].str.replace("10037", "Central Harlem - Morningside Heights", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10039", "Central Harlem - Morningside Heights", regex=False)

In [33]:
# East Harlem 25 23
# 10029, 10035

df_MI['Precinct'] = df_MI['Precinct'].replace("10029", "25, 23", regex=False) 
df_MI['Precinct'] = df_MI['Precinct'].replace("10035", "25, 23", regex=False) 

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10029", "East Harlem", regex=False)
df_MI['NeighName'] =df_MI['NeighName'].str.replace("10035", "East Harlem", regex=False)

In [34]:
# Upper West Side 20 24
# 10023, 10024, 10025

df_MI['Precinct'] = df_MI['Precinct'].replace("10023", "20, 24", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10024", "20, 24", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10025", "20, 24", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10023", "Upper West Side", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10024", "Upper West Side", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10025", "Upper West Side", regex=False)

In [35]:
# Upper East Side 19
# 10021, 10028, 10044, 10128, 10065

df_MI['Precinct'] = df_MI['Precinct'].replace("10021", "19", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10028", "19", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10044", "19", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10128", "19", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10065", "19", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10075", "19", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10021", "Upper East Side", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10028", "Upper East Side", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10044", "Upper East Side", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10128", "Upper East Side", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10065", "Upper East Side", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10075", "Upper East Side", regex=False)

In [36]:
# Chelsea - Clinton 10
# 10001, 10011, 10018, 10019, 10020, 10036

df_MI['Precinct'] = df_MI['Precinct'].replace("10001", "10", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10011", "10", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10018", "10", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10019", "10", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10020", "10", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10036", "10", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10001", "Chelsea - Clinton", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10011", "Chelsea - Clinton", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10018", "Chelsea - Clinton", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10019", "Chelsea - Clinton", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10020", "Chelsea - Clinton", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10036", "Chelsea - Clinton", regex=False)

In [37]:
# Gramercy Park - Murray Hill 13 17
# 10010, 10016, 10017, 10022

df_MI['Precinct'] = df_MI['Precinct'].replace("10010", "13, 17", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10016", "13, 17", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10017", "13, 17", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10022", "13, 17", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10010", "Gramercy Park - Murray Hill", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10016", "Gramercy Park - Murray Hill", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10017", "Gramercy Park - Murray Hill", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10022", "Gramercy Park - Murray Hill", regex=False)

In [38]:
# Greenwich Village - SoHo 6 
# 10012, 10013, 10014

df_MI['Precinct'] = df_MI['Precinct'].replace("10012", "6", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10013", "6", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10014", "6", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10012", "Greenwich Village - SoHo", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10013", "Greenwich Village - SoHo", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10014", "Greenwich Village - SoHo", regex=False)

In [39]:
# Union Square - Lower East Side 7 9
# 10002, 10003, 10009

df_MI['Precinct'] = df_MI['Precinct'].replace("10002", "7, 9", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10003", "7, 9", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10009", "7, 9", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10002", "Union Square - Lower East Side", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10003", "Union Square - Lower East Side", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10009", "Union Square - Lower East Side", regex=False)

In [40]:
# Lower Manhattan 1 5
# 10004, 10005, 10006, 10007, 10038, 10280
    
df_MI['Precinct'] = df_MI['Precinct'].replace("10004", "1, 5", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10005", "1, 5", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10006", "1, 5", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10007", "1, 5", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10038", "1, 5", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10280", "1, 5", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10004", "Lower Manhattan", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10005", "Lower Manhattan", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10006", "Lower Manhattan", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10007", "Lower Manhattan", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10038", "Lower Manhattan", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10280", "Lower Manhattan", regex=False)

In [41]:
# Long Island City - Astoria 108 114
# 11101, 11102, 11103, 11104, 11105, 11106

df_MI['Precinct'] = df_MI['Precinct'].replace("11101", "108, 114", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11102", "108, 114", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11103", "108, 114", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11104", "108, 114", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11105", "108, 114", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11106", "108, 114", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11101", "Long Island City - Astoria", regex=False)
df_MI['NeighName'] =df_MI['NeighName'].str.replace("11102", "Long Island City - Astoria", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11103", "Long Island City - Astoria", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11104", "Long Island City - Astoria", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11105", "Long Island City - Astoria", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11106", "Long Island City - Astoria", regex=False)

In [42]:
# West Queens 103
# 11368, 11369, 11370, 11372, 11373, 11377, 11378

df_MI['Precinct'] = df_MI['Precinct'].replace("11368", "103", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11369", "103", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11370", "103", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11372", "103", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11373", "103", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11377", "103", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11378", "103", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11368", "West Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11369", "West Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11370", "West Queens", regex=False)
df_MI['NeighName'] =df_MI['NeighName'].str.replace("11372", "West Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11373", "West Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11377", "West Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11378", "West Queens", regex=False)

In [43]:
# Flushing - Clearview 109
# 11354, 11355, 11356, 11357, 11358, 11359, 11360

df_MI['Precinct'] = df_MI['Precinct'].replace("11354", "109", regex=False)
df_MI['Precinct'] =df_MI['Precinct'].replace("11355", "109", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11356", "109", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11357", "109", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11358", "109", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11359", "109", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11360", "109", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11354", "Flushing - Clearview", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11355", "Flushing - Clearview", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11356", "Flushing - Clearview", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11357", "Flushing - Clearview", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11358", "Flushing - Clearview", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11359", "Flushing - Clearview", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11360", "Flushing - Clearview", regex=False)

In [44]:
# Bayside - Little Neck 111
# 11361, 11362, 11363, 11364

df_MI['Precinct'] = df_MI['Precinct'].replace("11361", "111", regex=False)
df_MI['Precinct'] =df_MI['Precinct'].replace("11362", "111", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11363", "111", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11364", "111", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11361", "Bayside - Little Neck", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11362", "Bayside - Little Neck", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11363", "Bayside - Little Neck", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11364", "Bayside - Little Neck", regex=False)

In [45]:
# Ridgewood - Forest Hills 112 104
# 11374, 11375, 11379, 11385

df_MI['Precinct'] = df_MI['Precinct'].replace("11374", "112, 104", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11375", "112, 104", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11379", "112, 104", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11385", "112, 104", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11374", "Ridgewood - Forest Hills", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11375", "Ridgewood - Forest Hills", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11379", "Ridgewood - Forest Hills", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11385", "Ridgewood - Forest Hills", regex=False)

In [46]:
# Fresh Meadows 107 111
# 11365, 11366, 11367

df_MI['Precinct'] = df_MI['Precinct'].replace("11365", "107, 111", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11366", "107, 111", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11367", "107, 111", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11365", "Fresh Meadows", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11366", "Fresh Meadows", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11367", "Fresh Meadows", regex=False)

In [47]:
# Southwest Queens 108
# 11414, 11415, 11416, 11417, 11418, 11419, 11420, 11421

df_MI['Precinct'] = df_MI['Precinct'].replace("11414", "108", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11415", "108", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11416", "108", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11417", "108", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11418", "108", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11419", "108", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11420", "108", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11421", "108", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11414", "Southwest Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11415", "Southwest Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11416", "Southwest Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11417", "Southwest Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11418", "Southwest Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11419", "Southwest Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11420", "Southwest Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11421", "Southwest Queens", regex=False)

In [48]:
# Jamaica 103 113
# 11412, 11423, 11432, 11433, 11434, 11435, 11436

df_MI['Precinct'] = df_MI['Precinct'].replace("11412", "103, 113", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11423", "103, 113", regex=False)
df_MI['Precinct'] =df_MI['Precinct'].replace("11432", "103, 113", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11433", "103, 113", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11434", "103, 113", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11435", "103, 113", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11436", "103, 113", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11412", "Jamaica", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11423", "Jamaica", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11432", "Jamaica", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11433", "Jamaica", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11434", "Jamaica", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11435", "Jamaica", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11436", "Jamaica", regex=False)

In [49]:
# Southeast Queens 105 113
# 11004, 11005, 11411, 11413, 11422, 11426, 11427, 11428, 11429

df_MI['Precinct'] = df_MI['Precinct'].replace("11004", "105, 113", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11005", "105, 113", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11411", "105, 113", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11413", "105, 113", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11422", "105, 113", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11426", "105, 113", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11427", "105, 113", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11428", "105, 113", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11429", "105, 113", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11004", "Southeast Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11005", "Southeast Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11411", "Southeast Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11413", "Southeast Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11422", "Southeast Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11426", "Southeast Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11427", "Southeast Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11428", "Southeast Queens", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11429", "Southeast Queens", regex=False)

In [50]:
# IMPORTANT precincts 105, 113 span a large area

In [51]:
# Rockaway 100 101
# 11691, 11692, 11693, 11694, 11695, 11697

df_MI['Precinct'] = df_MI['Precinct'].replace("11691", "100, 101", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11692", "100, 101", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11693", "100, 101", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11694", "100, 101", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11695", "100, 101", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("11697", "100, 101", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("11691", "Rockaway", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11692", "Rockaway", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11693", "Rockaway", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11694", "Rockaway", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11695", "Rockaway", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("11697", "Rockaway", regex=False)

In [52]:
# Port Richmond 121
# 10302, 10303, 10310

df_MI['Precinct'] =df_MI['Precinct'].replace("10302", "121", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10303", "121", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10310", "121", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10302", "Port Richmond", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10303", "Port Richmond", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10310", "Port Richmond", regex=False)

In [53]:
# Stapleton - St. George 120
# 10301, 10304, 10305

df_MI['Precinct'] = df_MI['Precinct'].replace("10301", "120", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10304", "120", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10305", "120", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10301", "Stapleton - St. George", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10304", "Stapleton - St. George", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10305", "Stapleton - St. George", regex=False)

In [54]:
# Willowbrook 121
# 10314

df_MI['Precinct'] = df_MI['Precinct'].replace("10314", "121", regex=False)
df_MI['NeighName'] =df_MI['NeighName'].str.replace("10314", "Willowbrook", regex=False)

In [55]:
# South Beach - Tottenville 122 123
# 10306, 10307, 10308, 10309, 10312

df_MI['Precinct'] = df_MI['Precinct'].replace("10306", "122, 123", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10307", "122, 123", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10308", "122, 123", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10309", "122, 123", regex=False)
df_MI['Precinct'] = df_MI['Precinct'].replace("10312", "122, 123", regex=False)

df_MI['NeighName'] = df_MI['NeighName'].str.replace("10306", "South Beach - Tottenville", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10307", "South Beach - Tottenville", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10308", "South Beach - Tottenville", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10309", "South Beach - Tottenville", regex=False)
df_MI['NeighName'] = df_MI['NeighName'].str.replace("10312", "South Beach - Tottenville", regex=False)

In [56]:
#Split Precinct column by comma

df_MI = pd.concat([df_MI, df_MI.Precinct.str.split(', ', expand=True)], axis=1)

In [57]:
df_MI = df_MI.rename(columns={0: 'Precinct_1', 1: 'Precinct_2', 2:'Precinct_3'})

In [58]:
df_MI.head(10)

,id,Zip Code,Median household income2019,Margin of Error,Precinct,NeighName,Precinct_1,Precinct_2,Precinct_3
1,8600000US06390,06390,61125,8799,06390,06390,06390,None,None
2,8600000US10001,10001,92840,8971,10,Chelsea - Clinton,10,None,None
3,8600000US10002,10002,36982,2384,"7, 9",Union Square - Lower East Side,7,9,None
4,8600000US10003,10003,118161,10235,"7, 9",Union Square - Lower East Side,7,9,None
5,8600000US10004,10004,190223,49426,"1, 5",Lower Manhattan,1,5,None
6,8600000US10005,10005,189702,15832,"1, 5",Lower Manhattan,1,5,None
7,8600000US10006,10006,179044,16237,"1, 5",Lower Manhattan,1,5,None
8,8600000US10007,10007,224063,45620,"1, 5",Lower Manhattan,1,5,None
9,8600000US10009,10009,63717,4878,"7, 9",Union Square - Lower East Side,7,9,None
10,8600000US10010,10010,132988,7749,"13, 17",Gramercy Park - Murray Hill,13,17,None


In [59]:
df_MI['Median household income2019'] = df_MI['Median household income2019'].replace("-", "", regex=False)

In [60]:
df_MI2 = df_MI[df_MI['Median household income2019'].notna()]

In [61]:
# extract for df w/ NeighName 
df_MI2 = df_MI[df_MI.NeighName.str.contains(r"[A-Za-z]", regex=True)]

In [62]:
df_MI2.to_excel("ZipCode_Income.xlsx",index=False)

In [63]:
# manually filling in median household income for zipcodes 10020, 11359

In [64]:
!ls

ACSDT5Y2019.B19013_data_with_overlays_2021-03-28T171056 2.xlsx
ACSDT5Y2019.B19013_data_with_overlays_2021-03-28T171056.csv
ACSDT5Y2019.B19013_metadata_2021-03-28T171056.csv
ACSDT5Y2019.B19013_table_title_2021-03-28T171056.txt
Burglary2021.csv
Burglary_NarrowDown.csv
GrandLarceny2021.csv
NYC_IncomeLevels.xlsx
NYC_MajorFelony_CrimeStats_New4.xlsx
OLD_df_MI_P1.csv
PCT_GrandLarceny.csv
ShootingsPercentChange.csv
VehicleTheft2021.csv
ZipCode_Income.xlsx
ZipCode_Income2.xlsx
ZipCode_Income3.csv
carTheft_Pct.csv
change_19_20.csv
change_20_21.csv
df_MI.csv
df_MI_P1.xlsx
df_MI_Precincts.xlsx
df_MI_Precincts_New.xlsx
dfinal_new
drop_19_20.csv
drop_20_21.csv
lowerIncome.csv
lowerMiddleIncome.csv
rise_19_20.csv
rise_20_21.csv
upperIncome.csv
upperMiddleIncome.csv
~$ACSDT5Y2019.B19013_data_with_overlays_2021-03-28T171056 2.xlsx


In [65]:
df_MI3 = pd.read_excel("ZipCode_Income2.xlsx")

In [66]:
# IMPORTANT
df_MI3.to_csv("ZipCode_Income3.csv", index=False)

In [67]:
df_MI3.shape

(177, 9)

## reading with df w/ zip code & population info

In [68]:
os.chdir("..")

In [69]:
os.chdir("Sources")

In [70]:
df_zclist = pd.read_csv("nyc_zip_borough_neighborhoods_pop.csv")

In [71]:
df_zclist.head()

,zip,borough,post_office,neighborhood,population,density
0,10001,Manhattan,"New York, NY",Chelsea and Clinton,21102,33959
1,10002,Manhattan,"New York, NY",Lower East Side,81410,92573
2,10003,Manhattan,"New York, NY",Lower East Side,56024,97188
3,10004,Manhattan,"New York, NY",Lower Manhattan,3089,5519
4,10005,Manhattan,"New York, NY",Lower Manhattan,7135,97048


In [72]:
df_zclist = df_zclist.rename(columns={'zip':'Zip Code'})

In [73]:
df_zclist.shape

(177, 6)

# Step 2: Income range by precinct

In [74]:
df_MI4 = df_MI3

In [75]:
df_MI4['Median household income2019'] = df_MI4['Median household income2019'].astype(str).astype(int)

In [76]:
df_MI4.head(1)

,id,Zip Code,Median household income2019,Margin of Error,Precinct,NeighName,Precinct_1,Precinct_2,Precinct_3
0,8600000US10001,10001,92840,8971,10,Chelsea - Clinton,10,NaN,NaN


In [77]:
df_MI4['Precinct_1'] = df_MI4['Precinct_1'].astype(str).astype(int)

In [78]:
P1 = df_MI4.groupby('Precinct_1').agg({'Median household income2019':['median', 'min', 'max']})
P1 = P1.reset_index()

In [79]:
P1 = P1.rename(columns={'Median household income2019': 'P1_household_income'})
P1

Precinct_1 P1_household_income                
                           median     min     max
0           1            186231.0   95702  224063
1           6            113191.0  106467  133501
2           7             63717.0   36982  118161
3          10            113138.0   92840  138272
4          13            132016.5  126628  150718
5          19            122569.0  104808  148441
6          20            132605.0   91624  143623
7          25             31759.5   29799   33720
8          26             46263.0   42278   59691
9          33             53660.0   49677   63093
10         41             25775.0   21447   29284
11         44             29579.0   27917   30349
12         45             56291.5   33498   96366
13         48             29136.0   27545   29767
14         49             59829.5   51951   65399
15         50             73565.0   58408   88722
16         52             37804.0   35423   39372
17         61             54334.5   32549   64631
18         66             53081.5   40683   73470
19         67             59038.5   54457   69385
20         68             72902.0   56911   77917
21         69             70793.0   29172   85807
22         72             55695.5   50191   61200
23         75             40892.5   39382   42403
24         77             44905.0   26521   91846
25         78            119375.0   58872  132091
26         83             53303.0   39753   56900
27         94             87358.0   81228   93488
28        100             60569.0   46819  105750
29        103             62927.0   54184   81608
30        105             88300.0   55313   98987
31        107             67642.0   63546   84301
32        108             74420.5   67751   79883
33        109             68848.0   40786   85818
34        111             90612.5   82028  101019
35        112             79611.5   66795   84713
36        120             62052.0   54199   79742
37        121             71645.0   66096   91505
38        122             97407.0   80834  109457

In [80]:
df_MI_1 = pd.merge(df_MI4, P1, on='Precinct_1')

/Users/lisakim/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/lisakim/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/core/generic.py:3887: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [81]:
df_MI_1.shape

(177, 12)

In [82]:
df_MI_1.to_excel("df_MI_P1.xlsx",index=False)

In [83]:
df_MI4.head()

,id,Zip Code,Median household income2019,Margin of Error,Precinct,NeighName,Precinct_1,Precinct_2,Precinct_3
0,8600000US10001,10001,92840,8971,10,Chelsea - Clinton,10,NaN,NaN
1,8600000US10002,10002,36982,2384,"7, 9",Union Square - Lower East Side,7,9.0,NaN
2,8600000US10003,10003,118161,10235,"7, 9",Union Square - Lower East Side,7,9.0,NaN
3,8600000US10004,10004,190223,49426,"1, 5",Lower Manhattan,1,5.0,NaN
4,8600000US10005,10005,189702,15832,"1, 5",Lower Manhattan,1,5.0,NaN


In [84]:
P2 = df_MI4.groupby('Precinct_2').agg({'Median household income2019':['median', 'min', 'max']})
P2 = P2.reset_index()

In [85]:
P2 = P2.rename(columns={'Median household income2019': 'P2_household_income'})
P2

Precinct_2 P2_household_income                
                           median     min     max
0         5.0            186231.0   95702  224063
1         9.0             63717.0   36982  118161
2        17.0            132016.5  126628  150718
3        23.0             31759.5   29799   33720
4        24.0            132605.0   91624  143623
5        28.0             46263.0   42278   59691
6        34.0             53660.0   49677   63093
7        40.0             25775.0   21447   29284
8        42.0             29136.0   27545   29767
9        45.0             59829.5   51951   65399
10       46.0             37804.0   35423   39372
11       62.0             72902.0   56911   77917
12       63.0             70793.0   29172   85807
13       68.0             50191.0   50191   50191
14       70.0             59038.5   54457   69385
15       76.0             61200.0   61200   61200
16       81.0             44905.0   26521   91846
17      101.0             60569.0   46819  105750
18      104.0             79611.5   66795   84713
19      111.0             67642.0   63546   84301
20      113.0             78876.5   54253   98987
21      114.0             73481.5   67751   79883
22      123.0             97407.0   80834  109457

In [86]:
P3 = df_MI4.groupby('Precinct_3').agg({'Median household income2019':['median', 'min', 'max']})
P3 = P3.reset_index()

In [87]:
P3 = P3.rename(columns={'Median household income2019': 'P3_household_income'})
P3

Precinct_3 P3_household_income              
                          median    min    max
0       71.0               44905  26521  91846

### Manually cleaned "df_MI_Precincts.xlsx" data

In [88]:
os.chdir("..")

In [89]:
os.chdir("MedianIncomeData")

In [90]:
!ls

ACSDT5Y2019.B19013_data_with_overlays_2021-03-28T171056 2.xlsx
ACSDT5Y2019.B19013_data_with_overlays_2021-03-28T171056.csv
ACSDT5Y2019.B19013_metadata_2021-03-28T171056.csv
ACSDT5Y2019.B19013_table_title_2021-03-28T171056.txt
Burglary2021.csv
Burglary_NarrowDown.csv
GrandLarceny2021.csv
NYC_IncomeLevels.xlsx
NYC_MajorFelony_CrimeStats_New4.xlsx
OLD_df_MI_P1.csv
PCT_GrandLarceny.csv
ShootingsPercentChange.csv
VehicleTheft2021.csv
ZipCode_Income.xlsx
ZipCode_Income2.xlsx
ZipCode_Income3.csv
carTheft_Pct.csv
change_19_20.csv
change_20_21.csv
df_MI.csv
df_MI_P1.xlsx
df_MI_Precincts.xlsx
df_MI_Precincts_New.xlsx
dfinal_new
drop_19_20.csv
drop_20_21.csv
lowerIncome.csv
lowerMiddleIncome.csv
rise_19_20.csv
rise_20_21.csv
upperIncome.csv
upperMiddleIncome.csv
~$ACSDT5Y2019.B19013_data_with_overlays_2021-03-28T171056 2.xlsx


In [91]:
# IMPORTANT Read cleaned data
# IMPORTANT precincts 77, 81, 71, 69 large income gap 
df_Precincts_Income_Final = pd.read_excel("df_MI_Precincts_New.xlsx")

In [92]:
# 'df_Precincts_Income_Final' includes median, min and max income by precinct
# most zip codes are divided in 2-4 precincts 

df_Precincts_Income_Final

,Precinct,median_of_household_incomes,min_household_income,max_household_income
0,1,186231.0,95702,224063
1,5,186231.0,95702,224063
2,6,113191.0,106467,133501
3,7,63717.0,36982,118161
4,9,63717.0,36982,118161
...,...,...,...,...
71,115,57435.0,54184,63706
72,120,62052.0,54199,79742
73,121,71645.0,66096,91505
74,122,97407.0,80834,109457


# Step 3: get annual data on 7 major felony offenses
## crime change by precinct focus on (2016, 2017, 2018, 2019, 2020)

In [93]:
os.chdir("..")

In [94]:
df_MF = pd.read_excel('seven-major-felony-offenses-annual-2000-2020.xls')

In [95]:
df_MF.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,Seven Major Felony Offenses,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OFFENSE,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
4,MURDER & NON-NEGL. MANSLAUGHTER,673.0,649.0,587.0,597.0,570.0,539.0,596.0,496.0,523.0,471.0,536.0,515.0,419.0,335.0,333.0,352.0,335.0,292.0,295.0,319.0,468.0
5,RAPE,2068.0,1981.0,2144.0,2070.0,1905.0,1858.0,1525.0,1351.0,1299.0,1205.0,1373.0,1420.0,1445.0,1378.0,1352.0,1438.0,1438.0,1449.0,1794.0,1755.0,1427.0
6,ROBBERY,32562.0,28202.0,27229.0,25989.0,24373.0,24722.0,23739.0,21809.0,22401.0,18601.0,19486.0,19717.0,20144.0,19128.0,16539.0,16931.0,15500.0,13956.0,12913.0,13371.0,13106.0
7,FELONY ASSAULT,25924.0,23453.0,21147.0,19139.0,18622.0,17750.0,17309.0,17493.0,16284.0,16773.0,16956.0,18482.0,19381.0,20297.0,20207.0,20270.0,20847.0,20052.0,20208.0,20698.0,20572.0
8,BURGLARY,38352.0,32763.0,31275.0,29110.0,26976.0,24117.0,23143.0,21762.0,20725.0,19430.0,18600.0,18720.0,19168.0,17429.0,16765.0,15125.0,12990.0,12083.0,11687.0,10783.0,15478.0
9,GRAND LARCENY,49631.0,46329.0,45771.0,46751.0,48763.0,48243.0,46625.0,44924.0,44242.0,39580.0,37835.0,38501.0,42497.0,45368.0,43862.0,44005.0,44279.0,43150.0,43558.0,43250.0,35505.0


In [96]:
new_header = df_MF.iloc[3] #grab the first row for the header
df_MF = df_MF[4:] #take the data less the header row
df_MF.columns = new_header #set the header row as the df header

In [97]:
df_MF.head(10)

3,OFFENSE,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
4,MURDER & NON-NEGL. MANSLAUGHTER,673.0,649.0,587.0,597.0,570.0,539.0,596.0,496.0,523.0,471.0,536.0,515.0,419.0,335.0,333.0,352.0,335.0,292.0,295.0,319.0,468.0
5,RAPE,2068.0,1981.0,2144.0,2070.0,1905.0,1858.0,1525.0,1351.0,1299.0,1205.0,1373.0,1420.0,1445.0,1378.0,1352.0,1438.0,1438.0,1449.0,1794.0,1755.0,1427.0
6,ROBBERY,32562.0,28202.0,27229.0,25989.0,24373.0,24722.0,23739.0,21809.0,22401.0,18601.0,19486.0,19717.0,20144.0,19128.0,16539.0,16931.0,15500.0,13956.0,12913.0,13371.0,13106.0
7,FELONY ASSAULT,25924.0,23453.0,21147.0,19139.0,18622.0,17750.0,17309.0,17493.0,16284.0,16773.0,16956.0,18482.0,19381.0,20297.0,20207.0,20270.0,20847.0,20052.0,20208.0,20698.0,20572.0
8,BURGLARY,38352.0,32763.0,31275.0,29110.0,26976.0,24117.0,23143.0,21762.0,20725.0,19430.0,18600.0,18720.0,19168.0,17429.0,16765.0,15125.0,12990.0,12083.0,11687.0,10783.0,15478.0
9,GRAND LARCENY,49631.0,46329.0,45771.0,46751.0,48763.0,48243.0,46625.0,44924.0,44242.0,39580.0,37835.0,38501.0,42497.0,45368.0,43862.0,44005.0,44279.0,43150.0,43558.0,43250.0,35505.0
10,GRAND LARCENY OF MOTOR VEHICLE,35442.0,29531.0,26656.0,23413.0,20884.0,18246.0,15745.0,13174.0,12482.0,10670.0,10329.0,9314.0,8093.0,7400.0,7664.0,7332.0,6327.0,5676.0,5428.0,5430.0,9037.0
11,TOTAL SEVEN MAJOR FELONY OFFENSES,184652.0,162908.0,154809.0,147069.0,142093.0,135475.0,128682.0,121009.0,117956.0,106730.0,105115.0,106669.0,111147.0,111335.0,106722.0,105453.0,101716.0,96658.0,95883.0,95606.0,95593.0
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
# select 2016, 2017, 2018, 2019, 2020

In [99]:
df_MF.columns

Index(['OFFENSE',    2000.0,    2001.0,    2002.0,    2003.0,    2004.0,
          2005.0,    2006.0,    2007.0,    2008.0,    2009.0,    2010.0,
          2011.0,    2012.0,    2013.0,    2014.0,    2015.0,    2016.0,
          2017.0,    2018.0,    2019.0,    2020.0],
      dtype='object', name=3)

In [100]:
df_MF2 = df_MF[['OFFENSE', 2015.0, 2016.0, 2017.0, 2018.0,2019.0, 2020.0]]

In [101]:
df_MF2 = df_MF2.reset_index(drop=True)

In [102]:
df_MF2 = df_MF2.loc[:7]

In [103]:
df_MF2

3,OFFENSE,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
0,MURDER & NON-NEGL. MANSLAUGHTER,352.0,335.0,292.0,295.0,319.0,468.0
1,RAPE,1438.0,1438.0,1449.0,1794.0,1755.0,1427.0
2,ROBBERY,16931.0,15500.0,13956.0,12913.0,13371.0,13106.0
3,FELONY ASSAULT,20270.0,20847.0,20052.0,20208.0,20698.0,20572.0
4,BURGLARY,15125.0,12990.0,12083.0,11687.0,10783.0,15478.0
5,GRAND LARCENY,44005.0,44279.0,43150.0,43558.0,43250.0,35505.0
6,GRAND LARCENY OF MOTOR VEHICLE,7332.0,6327.0,5676.0,5428.0,5430.0,9037.0
7,TOTAL SEVEN MAJOR FELONY OFFENSES,105453.0,101716.0,96658.0,95883.0,95606.0,95593.0


In [104]:
df_MF2 = df_MF2.set_index(['OFFENSE'])

In [105]:
df_MF2

3,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
OFFENSE,,,,,,
MURDER & NON-NEGL. MANSLAUGHTER,352.0,335.0,292.0,295.0,319.0,468.0
RAPE,1438.0,1438.0,1449.0,1794.0,1755.0,1427.0
ROBBERY,16931.0,15500.0,13956.0,12913.0,13371.0,13106.0
FELONY ASSAULT,20270.0,20847.0,20052.0,20208.0,20698.0,20572.0
BURGLARY,15125.0,12990.0,12083.0,11687.0,10783.0,15478.0
GRAND LARCENY,44005.0,44279.0,43150.0,43558.0,43250.0,35505.0
GRAND LARCENY OF MOTOR VEHICLE,7332.0,6327.0,5676.0,5428.0,5430.0,9037.0
TOTAL SEVEN MAJOR FELONY OFFENSES,105453.0,101716.0,96658.0,95883.0,95606.0,95593.0


In [106]:
# getting annual pct changes
df_MF3 = df_MF2.pct_change(axis='columns', periods=1)

In [107]:
df_MF3

3,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
OFFENSE,,,,,,
MURDER & NON-NEGL. MANSLAUGHTER,NaN,-0.048295,-0.128358,0.010274,0.081356,0.467085
RAPE,NaN,0.000000,0.007650,0.238095,-0.021739,-0.186895
ROBBERY,NaN,-0.084520,-0.099613,-0.074735,0.035468,-0.019819
FELONY ASSAULT,NaN,0.028466,-0.038135,0.007780,0.024248,-0.006088
BURGLARY,NaN,-0.141157,-0.069823,-0.032773,-0.077351,0.435408
GRAND LARCENY,NaN,0.006227,-0.025497,0.009455,-0.007071,-0.179075
GRAND LARCENY OF MOTOR VEHICLE,NaN,-0.137070,-0.102892,-0.043693,0.000368,0.664273
TOTAL SEVEN MAJOR FELONY OFFENSES,NaN,-0.035438,-0.049727,-0.008018,-0.002889,-0.000136


In [108]:
df_MF3 = df_MF3 * 100

In [109]:
df_MF3

3,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
OFFENSE,,,,,,
MURDER & NON-NEGL. MANSLAUGHTER,NaN,-4.829545,-12.835821,1.027397,8.135593,46.708464
RAPE,NaN,0.000000,0.764951,23.809524,-2.173913,-18.689459
ROBBERY,NaN,-8.451952,-9.961290,-7.473488,3.546813,-1.981901
FELONY ASSAULT,NaN,2.846571,-3.813498,0.777977,2.424782,-0.608754
BURGLARY,NaN,-14.115702,-6.982294,-3.277332,-7.735090,43.540759
GRAND LARCENY,NaN,0.622657,-2.549741,0.945539,-0.707103,-17.907514
GRAND LARCENY OF MOTOR VEHICLE,NaN,-13.707038,-10.289237,-4.369274,0.036846,66.427256
TOTAL SEVEN MAJOR FELONY OFFENSES,NaN,-3.543759,-4.972669,-0.801796,-0.288894,-0.013597


In [110]:
# IMPORTANT
df_MF3.to_csv('seven-major-felony-offenses-annual-pct-change.csv')

# Step 4: Analyze major felony 2019, 2020, 2021.08.01
### Crimes changed -0.4% YoY between Jan.-July 2021 

In [111]:
#os.chdir("Master's Project")

In [271]:
df_MF4 = pd.read_excel('NYC_MajorFelony_CrimeStats_finalUpdate.xlsx')

In [273]:
df_MF4[df_MF4.PCT == 50]

,PCT,household_median_income,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
256,50,73565.0,MURDER & NON NEGL. MANSLAUGHTER,1,3,200.0,0,2,-200.000000
257,50,73565.0,RAPE,15,12,-20.0,11,8,37.500000
258,50,73565.0,ROBBERY,107,88,-17.8,50,40,25.000000
259,50,73565.0,FELONY ASSAULT,144,158,9.7,82,96,-14.583333
260,50,73565.0,BURGLARY,102,150,47.1,52,70,-25.714286
261,50,73565.0,GRAND LARCENY,500,461,-7.8,239,232,3.017241
262,50,73565.0,GRAND LARCENY OF MOTOR VEHICLE,90,152,68.9,60,80,-25.000000
263,50,73565.0,TOTAL SEVEN MAJOR FELONY OFFENSES,959,1024,6.8,494,528,-6.439394


In [114]:
#df_MF4['%_Chg'] = (df_MF4['2021_YTD'] - df_MF4['2020_YTD']) / df_MF4['2020_YTD'] * 100

In [115]:
df_MF4['%_Chg'] = round(df_MF4['%_Chg'],1)

In [116]:
(df_MF4['2021_YTD'].sum() - df_MF4['2020_YTD'].sum()) / df_MF4['2020_YTD'].sum() * 100

-0.3873528345450035

### new analysis Aug.8

In [117]:
lowerFinal = df_MF4[df_MF4.household_median_income < 68920.0]

In [118]:
lowerFinal.head(1)

,PCT,household_median_income,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
24,7,63717.0,MURDER & NON NEGL. MANSLAUGHTER,0,2,200.0,0,1,-100.0


In [119]:
# Groupby CRIME and give me the sum of numbers of columns 2019, 2020, 2021_YTD and 2020_YTD
# df.groupby(by=["b"]).sum()

In [120]:
lowerFinal_groupby = lowerFinal.groupby(by=["CRIME"]).sum()

In [121]:
# cols = [1,2,3,4,5,6]
# lowerFinal_groupby.drop(lowerFinal_groupby.columns[cols],axis=1,inplace=True)

In [122]:
lowerFinal_groupby

,PCT,household_median_income,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
CRIME,,,,,,,,
BURGLARY,2800,2250615.5,6946,9346,2319.0,4102,5410,-1106.2
FELONY ASSAULT,2800,2250615.5,15471,15225,-66.7,9226,8764,246.9
GRAND LARCENY,2800,2250615.5,21743,19377,-497.8,10839,10510,117.3
GRAND LARCENY OF MOTOR VEHICLE,2800,2250615.5,3498,5849,3504.3,3679,2826,1263.4
MURDER & NON NEGL. MANSLAUGHTER,2800,2250615.5,245,377,4001.2,194,206,1199.1
RAPE,2800,2250615.5,1235,1008,-283.2,577,584,365.1
ROBBERY,2800,2250615.5,9874,9486,-196.2,5116,5145,56.3
TOTAL SEVEN MAJOR FELONY OFFENSES,2800,2250615.5,59012,60668,111.0,33733,33445,-11.1


In [123]:
#lowerFinal_groupby = lowerFinal_groupby.drop(columns=['household_median_income'])

In [124]:
lowerFinal_groupby['%_chg_2019-2020'] = (lowerFinal_groupby[2020] - lowerFinal_groupby[2019])/lowerFinal_groupby[2019]*100

In [125]:
lowerFinal_groupby['%_chg_2019-2020'] = round(lowerFinal_groupby['%_chg_2019-2020'], 1)

In [126]:
lowerFinal_groupby['%_Chg'] = (lowerFinal_groupby['2021_YTD'] - lowerFinal_groupby['2020_YTD'])/lowerFinal_groupby['2020_YTD']*100

In [127]:
lowerFinal_groupby['%_Chg'] = round(lowerFinal_groupby['%_Chg'],1)

In [128]:
lowerFinal_groupby

,PCT,household_median_income,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
CRIME,,,,,,,,
BURGLARY,2800,2250615.5,6946,9346,34.6,4102,5410,-24.2
FELONY ASSAULT,2800,2250615.5,15471,15225,-1.6,9226,8764,5.3
GRAND LARCENY,2800,2250615.5,21743,19377,-10.9,10839,10510,3.1
GRAND LARCENY OF MOTOR VEHICLE,2800,2250615.5,3498,5849,67.2,3679,2826,30.2
MURDER & NON NEGL. MANSLAUGHTER,2800,2250615.5,245,377,53.9,194,206,-5.8
RAPE,2800,2250615.5,1235,1008,-18.4,577,584,-1.2
ROBBERY,2800,2250615.5,9874,9486,-3.9,5116,5145,-0.6
TOTAL SEVEN MAJOR FELONY OFFENSES,2800,2250615.5,59012,60668,2.8,33733,33445,0.9


In [129]:
lowerFinal_groupby.to_csv('lowerIncomeFinal_crimeTrend_final.csv', index=True)

In [130]:
#lowerMiddle = df_MF4[(df_MF4.household_median_income > 54103.6) & (df_MF4.household_median_income < 68920.0)]

In [131]:
#lowerMiddle_groupby = lowerMiddle.groupby(by=["CRIME"]).sum()

In [132]:
#lowerMiddle_groupby = lowerMiddle_groupby.drop(columns=['PCT', 'household_median_income'])

In [133]:
#lowerMiddle_groupby['%_chg_2019-2020'] = (lowerMiddle_groupby[2020] - lowerMiddle_groupby[2019])/lowerMiddle_groupby[2019]*100

In [134]:
#lowerMiddle_groupby['%_chg_2019-2020'] = round(lowerMiddle_groupby['%_chg_2019-2020'], 1)

In [135]:
#lowerMiddle_groupby['%_Chg'] = (lowerMiddle_groupby['2021_YTD'] - lowerMiddle_groupby['2020_YTD'])/lowerMiddle_groupby['2020_YTD']*100

In [136]:
#lowerMiddle_groupby['%_Chg'] = round(lowerMiddle_groupby['%_Chg'],1)

In [137]:
#lowerMiddle_groupby.to_csv('lowerMiddleIncome.csv', index=True)

In [138]:
#upperMiddle = df_MF4[(df_MF4.household_median_income > 68920.1) & (df_MF4.household_median_income < 91735.0)]

In [139]:
#upperMiddle_groupby = upperMiddle.groupby(by=["CRIME"]).sum()

In [140]:
#upperMiddle_groupby = upperMiddle_groupby.drop(columns=['PCT', 'household_median_income'])

In [141]:
#upperMiddle_groupby['%_chg_2019-2020'] = (upperMiddle_groupby[2020] - upperMiddle_groupby[2019])/upperMiddle_groupby[2019]*100

In [142]:
#upperMiddle_groupby['%_chg_2019-2020'] = round(upperMiddle_groupby['%_chg_2019-2020'], 1)

In [143]:
#upperMiddle_groupby['%_Chg'] = (upperMiddle_groupby['2021_YTD'] - upperMiddle_groupby['2020_YTD'])/upperMiddle_groupby['2020_YTD']*100

In [144]:
#upperMiddle_groupby['%_Chg'] = round(upperMiddle_groupby['%_Chg'],1)

In [145]:
#upperMiddle_groupby.to_csv('upperMiddleIncome.csv', index=True)

In [146]:
upperFinal = df_MF4[(df_MF4.household_median_income > 68920.1)]

In [147]:
upperFinal_groupby = upperFinal.groupby(by=["CRIME"]).sum()

In [148]:
upperFinal_groupby

,PCT,household_median_income,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
CRIME,,,,,,,,
BURGLARY,2076,3144514.0,3837,6132,1929.0,2569,3334,-578.1
FELONY ASSAULT,2076,3144514.0,5025,5285,98.9,3233,3013,359.8
GRAND LARCENY,2076,3144514.0,21506,16126,-588.3,8662,8796,40.9
GRAND LARCENY OF MOTOR VEHICLE,2076,3144514.0,1932,3188,2485.1,1707,1560,843.9
MURDER & NON NEGL. MANSLAUGHTER,2076,3144514.0,74,91,3046.7,51,45,915.0
RAPE,2076,3144514.0,519,418,-504.4,264,233,1086.9
ROBBERY,2076,3144514.0,3496,3620,217.7,1985,1981,47.5
TOTAL SEVEN MAJOR FELONY OFFENSES,2076,3144514.0,36389,34860,-26.2,18471,18962,-47.7


In [149]:
#upperFinal_groupby = upperFinal_groupby.drop(columns=['PCT', 'household_median_income'])

In [150]:
upperFinal_groupby['%_chg_2019-2020'] = (upperFinal_groupby[2020] - upperFinal_groupby[2019])/upperFinal_groupby[2019]*100

In [151]:
upperFinal_groupby['%_chg_2019-2020'] = round(upperFinal_groupby['%_chg_2019-2020'], 1)

In [152]:
upperFinal_groupby['%_Chg'] = (upperFinal_groupby['2021_YTD'] - upperFinal_groupby['2020_YTD'])/upperFinal_groupby['2020_YTD']*100

In [153]:
upperFinal_groupby['%_Chg'] = round(upperFinal_groupby['%_Chg'],1)

In [154]:
upperFinal_groupby.to_csv('upperIncome_crimeTrend_final.csv', index=True)

In [155]:
#os.chdir("Master's Project")

In [156]:
changeInCrimes_IncomeTier = pd.read_excel("changeInCrimes_IncomeTier.xlsx")

In [157]:
changeInCrimes_IncomeTier

,INCOME TIER,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
0,Lower Income,BURGLARY,6946,9346,34.6,4102,5410,-24.2
1,NaN,FELONY ASSAULT,15471,15225,-1.6,9226,8764,5.3
2,NaN,GRAND LARCENY,21743,19377,-10.9,10839,10510,3.1
3,NaN,GRAND LARCENY OF MOTOR VEHICLE,3498,5849,67.2,3679,2826,30.2
4,NaN,MURDER & NON NEGL. MANSLAUGHTER,245,377,53.9,194,206,-5.8
5,NaN,RAPE,1235,1008,-18.4,577,584,-1.2
6,NaN,ROBBERY,9874,9486,-3.9,5116,5145,-0.6
7,NaN,TOTAL SEVEN MAJOR FELONY OFFENSES,59012,60668,2.8,33733,33445,0.9
8,Upper Income,BURGLARY,3837,6132,59.8,2569,3334,-22.9
9,NaN,FELONY ASSAULT,5025,5285,5.2,3233,3013,7.3


In [158]:
citywideCrimeData = pd.read_excel("citywide_sevenMajor_Shooting_final.xlsx")

In [159]:
citywideCrimeData

,Crimes,2021_YTD,2020_YTD,%_chg
0,Murder,245,251,-2.390438
1,Rape,842,817,3.059976
2,Robbery,7101,7126,-0.350828
3,Felony Assault,12481,11793,5.833969
4,Burglary,6671,8744,-23.707685
5,Grand Larceny,19502,19307,1.009996
6,Vehicle Theft,5386,4386,22.799818
7,Shootings_h2,902,691,30.500000


In [160]:
citywideCrimeData.Crimes = citywideCrimeData.Crimes.str.replace("Shootings_h2", "Shootings (Jan.-June)")

In [161]:
citywideCrimeData['%_chg'] = round(citywideCrimeData['%_chg'], 1)

In [162]:
citywideCrimeData = citywideCrimeData.rename(columns={"%_chg": "Percent_Change"})

In [163]:
import altair as alt
from vega_datasets import data


alt.Chart(citywideCrimeData).mark_bar().encode(
    x="Crimes",
    y="Percent_Change:Q",
    color=alt.condition(
        alt.datum['Percent_Change'] > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange")  # The negative color
    )
).properties(width=400, height = 350, title ="How major crimes changed in NYC, Jan.-July 2021")

alt.Chart(...)

In [164]:
# making a modified version of df_MF4 for D3 line bubble graph
df_MF4

,PCT,household_median_income,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
0,1,186231.0,MURDER & NON NEGL. MANSLAUGHTER,1,1,0.0,1,0,100.0
1,1,186231.0,RAPE,15,19,26.7,10,14,-28.6
2,1,186231.0,ROBBERY,78,151,93.6,58,83,-30.1
3,1,186231.0,FELONY ASSAULT,98,81,-17.3,69,52,32.7
4,1,186231.0,BURGLARY,131,247,88.5,69,169,-59.2
...,...,...,...,...,...,...,...,...,...
611,123,97407.0,FELONY ASSAULT,62,59,-4.8,19,31,-38.7
612,123,97407.0,BURGLARY,32,41,28.1,16,29,-44.8
613,123,97407.0,GRAND LARCENY,170,178,4.7,94,100,-6.0
614,123,97407.0,GRAND LARCENY OF MOTOR VEHICLE,25,45,80.0,22,31,-29.0


In [165]:
df_d3 =df_MF4.copy(deep=True)

In [166]:
df_d3.CRIME = df_d3.CRIME.str.replace('MURDER & NON NEGL. MANSLAUGHTER','Murder')

In [167]:
df_d3.CRIME = df_d3.CRIME.str.replace('BURGLARY','Burglary')

In [168]:
df_d3.CRIME = df_d3.CRIME.str.replace('RAPE','Rape').str.replace('ROBBERY','Robbery')

In [169]:
df_d3.CRIME = df_d3.CRIME.str.replace('FELONY ASSAULT','Felony Assault').str.replace('GRAND LARCENY','Grand Larceny')

In [170]:
df_d3.CRIME = df_d3.CRIME.str.replace('Grand Larceny OF MOTOR VEHICLE','Vehicle Theft').str.replace('TOTAL SEVEN MAJOR FELONY OFFENSES','Total Seven Major Felonies')

In [171]:
col_list = ['PCT', 'household_median_income', 'CRIME', '%_chg_2019-2020','%_Chg']

df_d3 = df_d3[col_list]

df_d3

,PCT,household_median_income,CRIME,%_chg_2019-2020,%_Chg
0,1,186231.0,Murder,0.0,100.0
1,1,186231.0,Rape,26.7,-28.6
2,1,186231.0,Robbery,93.6,-30.1
3,1,186231.0,Felony Assault,-17.3,32.7
4,1,186231.0,Burglary,88.5,-59.2
...,...,...,...,...,...
611,123,97407.0,Felony Assault,-4.8,-38.7
612,123,97407.0,Burglary,28.1,-44.8
613,123,97407.0,Grand Larceny,4.7,-6.0
614,123,97407.0,Vehicle Theft,80.0,-29.0


In [172]:
df_d3 = df_d3.rename(columns = {'%_Chg':'Chg2021', '%_chg_2019-2020':'Chg2020'})

In [173]:
df_d3.head(1)

,PCT,household_median_income,CRIME,Chg2020,Chg2021
0,1,186231.0,Murder,0.0,100.0


In [174]:
df_d3.CRIME = df_d3.CRIME.astype(str)

In [175]:
D3_noMurder = df_d3[df_d3.CRIME.str.strip() != 'Murder']

In [176]:
D3_noMurder[D3_noMurder.Chg2021 ==430]

,PCT,household_median_income,CRIME,Chg2020,Chg2021
94,20,132605.0,Vehicle Theft,15.8,430.0


In [177]:
#df_d3.to_csv("D3BubbleLineGraph.csv", index=False)

In [178]:
#D3_noMurder.to_csv("D3_noMuder_new.csv", index=False)

In [179]:
df_d3 = df_d3[df_d3.household_median_income != 0]

In [180]:
df_d3 = df_d3[df_d3.CRIME.str.strip() != 'Total Seven Major Felonies']

In [181]:
df_d3.household_median_income.min()

25775.0

In [182]:
df_d3.head(1)

,PCT,household_median_income,CRIME,Chg2020,Chg2021
0,1,186231.0,Murder,0.0,100.0


In [183]:
col_list = ['PCT', 'household_median_income','CRIME','Chg2021']
df_d3_2021 = df_d3[col_list]

In [184]:
df_d3_2021.shape

(532, 4)

In [185]:
dfinal_new2 = pd.read_csv("dfinal_new2.csv")

In [186]:
# dfinal_new2 -- shooting data by precinct from the "Shootings" section
# combine dfinal_new2 with df_d3_2021

frames = [dfinal_new2, df_d3_2021]
result2021 = pd.concat(frames)

In [187]:
citywideCrimeData.Crimes=citywideCrimeData.Crimes.replace("Shootings (Jan.-June)", "Shooting (Jan.-June)")

In [188]:
citywideCrimeData = citywideCrimeData.rename(columns={"Crimes": "Crime"})

In [189]:
# IMPORTANT

In [190]:
import altair as alt
from vega_datasets import data

points = alt.Chart(result2021).mark_circle(
    opacity=0.5,
    stroke='black',
    strokeWidth=0.7
).encode(
    alt.X('Chg2021:Q', axis=alt.Axis(labelAngle=0), title ="Percent change (each dot represents a precinct)"),
    alt.Y('CRIME', title="Crime"),
    alt.Size('household_median_income:Q',
        scale=alt.Scale(range=[50, 3000]),
        legend=alt.Legend(title='Median Household Income')
    ),
    alt.Color('CRIME', legend=None)
).properties(
    width=800,
    height=500
)

In [191]:
bars = alt.Chart(citywideCrimeData).mark_bar(opacity=0.8).encode(
    alt.X("Percent_Change:Q", title = "Percent change"),
    y="Crime",
    color=alt.condition(
        alt.datum['Percent_Change'] > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange")  # The negative color
    )
).properties(width=800, height = 150)

In [192]:
alt.vconcat(
    points,
    bars,
    title="How major crimes changed in NYC, Jan.-July 2021"
)

alt.VConcatChart(...)

## Change in crime by income tier

In [ ]:
# Lower income

In [290]:
lowerChange = df_MF4[df_MF4.household_median_income < 68920]

In [291]:
lowerChange = lowerChange.groupby("CRIME").sum()

In [302]:
# important
lowerChange['%_chg_2019-2020'] = (lowerChange[2020] - lowerChange[2019]) / lowerChange[2019] * 100

In [303]:
lowerChange['%_chg_2019-2020'] = round(lowerChange['%_chg_2019-2020'], 1)

In [305]:
lowerChange['%_Chg'] = (lowerChange['2021_YTD'] - lowerChange['2020_YTD']) / lowerChange['2020_YTD'] * 100

In [306]:
lowerChange['%_Chg'] = round(lowerChange['%_Chg'], 1)

In [307]:
lowerChange

,PCT,household_median_income,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
CRIME,,,,,,,,
BURGLARY,2800,2250615.5,6946,9346,34.6,4102,5410,-24.2
FELONY ASSAULT,2800,2250615.5,15471,15225,-1.6,9226,8764,5.3
GRAND LARCENY,2800,2250615.5,21743,19377,-10.9,10839,10510,3.1
GRAND LARCENY OF MOTOR VEHICLE,2800,2250615.5,3498,5849,67.2,3679,2826,30.2
MURDER & NON NEGL. MANSLAUGHTER,2800,2250615.5,245,377,53.9,194,206,-5.8
RAPE,2800,2250615.5,1235,1008,-18.4,577,584,-1.2
ROBBERY,2800,2250615.5,9874,9486,-3.9,5116,5145,-0.6
TOTAL SEVEN MAJOR FELONY OFFENSES,2800,2250615.5,59012,60668,2.8,33733,33445,0.9


In [293]:
# Upper income

In [294]:
upperChange = df_MF4[df_MF4.household_median_income > 68920]

In [295]:
upperChange = upperChange.groupby("CRIME").sum()

In [308]:
upperChange['%_chg_2019-2020']=(upperChange[2020] - upperChange[2019]) / upperChange[2019] * 100

In [309]:
upperChange['%_chg_2019-2020']= round(upperChange['%_chg_2019-2020'],1)

In [310]:
upperChange['%_Chg'] = (upperChange['2021_YTD'] - upperChange['2020_YTD']) / upperChange['2020_YTD'] * 100

In [311]:
upperChange['%_Chg'] = round(upperChange['%_Chg'], 1)

In [312]:
upperChange

,PCT,household_median_income,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
CRIME,,,,,,,,
BURGLARY,2076,3144514.0,3837,6132,59.8,2569,3334,-22.9
FELONY ASSAULT,2076,3144514.0,5025,5285,5.2,3233,3013,7.3
GRAND LARCENY,2076,3144514.0,21506,16126,-25.0,8662,8796,-1.5
GRAND LARCENY OF MOTOR VEHICLE,2076,3144514.0,1932,3188,65.0,1707,1560,9.4
MURDER & NON NEGL. MANSLAUGHTER,2076,3144514.0,74,91,23.0,51,45,13.3
RAPE,2076,3144514.0,519,418,-19.5,264,233,13.3
ROBBERY,2076,3144514.0,3496,3620,3.5,1985,1981,0.2
TOTAL SEVEN MAJOR FELONY OFFENSES,2076,3144514.0,36389,34860,-4.2,18471,18962,-2.6


### murder

In [193]:
df_MF4.CRIME = df_MF4.CRIME.str.strip()

In [194]:
df_murder = df_MF4[df_MF4.CRIME == "MURDER & NON NEGL. MANSLAUGHTER"]

In [195]:
df_MF4.head(1)

,PCT,household_median_income,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
0,1,186231.0,MURDER & NON NEGL. MANSLAUGHTER,1,1,0.0,1,0,100.0


In [196]:
(df_murder['2021_YTD'].sum() - df_murder['2020_YTD'].sum()) / df_murder['2020_YTD'].sum()*100

-2.3904382470119523

In [197]:
df_murder['%_chg_2019-2020'] = round(df_murder['%_chg_2019-2020'],1)

<ipython-input-197-511bba0d13db>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_murder['%_chg_2019-2020'] = round(df_murder['%_chg_2019-2020'],1)


In [198]:
df_murder['%_chg_2019-2020'] = df_murder['%_chg_2019-2020'].fillna(0.0).astype(int)

<ipython-input-198-929872bd3cc4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_murder['%_chg_2019-2020'] = df_murder['%_chg_2019-2020'].fillna(0.0).astype(int)


In [199]:
df_murder['%_Chg'] = round(df_murder['%_Chg'],1)

<ipython-input-199-16d18dcb172e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_murder['%_Chg'] = round(df_murder['%_Chg'],1)


In [200]:
import statistics

In [201]:
statistics.stdev(df_murder['%_chg_2019-2020'])

151.10092682991228

In [202]:
statistics.stdev(df_murder['%_Chg'])

124.23815361667549

In [203]:
df_murder[(df_murder['%_chg_2019-2020'] > -28.5) & (df_murder['%_chg_2019-2020'] < 122.5)]

,PCT,household_median_income,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
0,1,186231.0,MURDER & NON NEGL. MANSLAUGHTER,1,1,0,1,0,100.0
16,6,113191.0,MURDER & NON NEGL. MANSLAUGHTER,0,1,100,0,0,0.0
64,17,132016.5,MURDER & NON NEGL. MANSLAUGHTER,0,0,0,0,0,0.0
80,19,122569.0,MURDER & NON NEGL. MANSLAUGHTER,0,1,100,1,1,0.0
88,20,132605.0,MURDER & NON NEGL. MANSLAUGHTER,1,1,0,0,0,0.0
128,26,46263.0,MURDER & NON NEGL. MANSLAUGHTER,2,2,0,1,1,0.0
136,28,46263.0,MURDER & NON NEGL. MANSLAUGHTER,5,7,40,2,4,-50.0
144,30,53660.0,MURDER & NON NEGL. MANSLAUGHTER,4,4,0,1,4,-75.0
152,32,46263.0,MURDER & NON NEGL. MANSLAUGHTER,10,9,-10,8,3,166.7
160,33,53660.0,MURDER & NON NEGL. MANSLAUGHTER,4,4,0,2,1,100.0


In [204]:
df_murder.sort_values(by='%_Chg', ascending=False).head(2)

,PCT,household_median_income,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
408,81,44905.0,MURDER & NON NEGL. MANSLAUGHTER,3,8,166,8,1,700.0
328,69,70793.0,MURDER & NON NEGL. MANSLAUGHTER,2,4,100,6,1,500.0


In [205]:
df_murder[df_murder['%_Chg'] < -2.4]

,PCT,household_median_income,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
24,7,63717.0,MURDER & NON NEGL. MANSLAUGHTER,0,2,200,0,1,-100.0
40,10,113138.0,MURDER & NON NEGL. MANSLAUGHTER,0,3,300,1,2,-50.0
48,13,132016.5,MURDER & NON NEGL. MANSLAUGHTER,0,3,300,1,2,-50.0
104,23,31759.5,MURDER & NON NEGL. MANSLAUGHTER,2,9,350,4,7,-42.9
112,24,132605.0,MURDER & NON NEGL. MANSLAUGHTER,2,8,300,0,8,-100.0
120,25,31759.5,MURDER & NON NEGL. MANSLAUGHTER,3,12,300,6,8,-25.0
136,28,46263.0,MURDER & NON NEGL. MANSLAUGHTER,5,7,40,2,4,-50.0
144,30,53660.0,MURDER & NON NEGL. MANSLAUGHTER,4,4,0,1,4,-75.0
232,47,58393.0,MURDER & NON NEGL. MANSLAUGHTER,8,12,50,5,8,-37.5
248,49,59829.5,MURDER & NON NEGL. MANSLAUGHTER,3,6,100,2,4,-50.0


In [206]:
len(df_murder[df_murder['%_Chg'] > -2.4])

50

In [207]:
df_murder[df_murder['%_Chg'] > -2.4].household_median_income.sum()/50

74734.71

In [208]:
df_murder[df_murder['%_Chg'] > -2.4].household_median_income.median()

70793.0

In [209]:
len(df_murder[df_murder['%_Chg'] < -2.4])

27

In [210]:
df_murder[df_murder['%_Chg'] < -2.4].household_median_income.sum()/27

61422.0

In [211]:
df_murder[df_murder['%_Chg'] < -2.4].household_median_income.median()

58393.0

### rape

In [212]:
df_rape = df_MF4[df_MF4.CRIME.str.strip() == "RAPE"]

In [213]:
((df_rape['2021_YTD'].sum() - df_rape['2020_YTD'].sum()) / df_rape['2020_YTD'].sum())*100 

2.937576499388005

In [214]:
df_rape[df_rape['%_Chg'] > 3]

,PCT,household_median_income,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
17,6,113191.0,RAPE,6,7,16.7,6,4,50.0
25,7,63717.0,RAPE,7,14,100.0,8,7,14.3
33,9,63717.0,RAPE,15,10,-33.3,12,5,140.0
41,10,113138.0,RAPE,15,6,-60.0,8,6,33.3
49,13,132016.5,RAPE,17,13,-23.5,16,5,220.0
73,18,99546.5,RAPE,29,14,-51.7,10,6,66.7
81,19,122569.0,RAPE,18,12,-33.3,7,6,16.7
89,20,132605.0,RAPE,10,5,-50.0,5,3,66.7
97,22,0.0,RAPE,1,2,100.0,5,2,150.0
105,23,31759.5,RAPE,20,17,-15.0,14,10,40.0


In [215]:
len(df_rape[df_rape['%_Chg'] > 3])

41

In [216]:
#df_rape[df_rape['%_Chg'] > 3].household_median_income.sum()/39

In [217]:
df_rape[df_rape['%_Chg'] > 3].household_median_income.median()

63717.0

In [218]:
len(df_rape[df_rape['%_Chg'] < 3])

36

In [219]:
#df_rape[df_rape['%_Chg'] < 3].household_median_income.sum()/36

In [220]:
df_rape[df_rape['%_Chg'] < 3].household_median_income.median()

58632.5

### robbery

In [221]:
df_robbery = df_MF4[df_MF4.CRIME.str.strip() == "ROBBERY"]

In [222]:
((df_robbery['2021_YTD'].sum() - df_robbery['2020_YTD'].sum()) / df_robbery['2020_YTD'].sum())*100 

-0.3508279539713724

In [223]:
df_robbery[df_robbery['%_Chg']>-0.4]

,PCT,household_median_income,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
18,6,113191.0,ROBBERY,143,142,-0.7,98,68,44.1
34,9,63717.0,ROBBERY,119,94,-21.0,82,54,51.9
42,10,113138.0,ROBBERY,123,126,2.4,71,71,0.0
58,14,95301.0,ROBBERY,142,174,22.5,214,76,181.6
66,17,132016.5,ROBBERY,60,64,6.7,34,29,17.2
90,20,132605.0,ROBBERY,85,90,5.9,58,43,34.9
98,22,0.0,ROBBERY,21,10,-52.4,9,4,125.0
106,23,31759.5,ROBBERY,168,168,0.0,104,91,14.3
114,24,132605.0,ROBBERY,172,136,-20.9,83,82,1.2
138,28,46263.0,ROBBERY,163,143,-12.3,88,73,20.5


In [224]:
len(df_robbery[df_robbery['%_Chg']>-0.4])

33

In [225]:
#df_robbery[df_robbery['%_Chg'] > -0.94].household_median_income.sum()/32

In [226]:
df_robbery[df_robbery['%_Chg'] > -0.4].household_median_income.median()

60569.0

In [227]:
len(df_robbery[df_robbery['%_Chg']<-0.4])

44

In [228]:
# df_robbery[df_robbery['%_Chg'] < -0.94].household_median_income.sum()/45

In [229]:
df_robbery[df_robbery['%_Chg'] < -0.4].household_median_income.median()

61626.0

### felony assault

In [230]:
df_felonyassault = df_MF4[df_MF4.CRIME.str.strip() == "FELONY ASSAULT"]

In [231]:
((df_felonyassault['2021_YTD'].sum() - df_felonyassault['2020_YTD'].sum()) / df_felonyassault['2020_YTD'].sum())*100 

5.79094845886049

In [232]:
df_felonyassault[df_felonyassault['%_Chg'] > 5.8]

,PCT,household_median_income,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
3,1,186231.0,FELONY ASSAULT,98,81,-17.3,69,52,32.7
19,6,113191.0,FELONY ASSAULT,135,152,12.6,101,95,6.3
27,7,63717.0,FELONY ASSAULT,187,122,-34.8,101,75,34.7
43,10,113138.0,FELONY ASSAULT,105,109,3.8,67,60,11.7
59,14,95301.0,FELONY ASSAULT,213,223,4.7,268,96,179.2
67,17,132016.5,FELONY ASSAULT,96,66,-31.2,67,35,91.4
75,18,99546.5,FELONY ASSAULT,162,126,-22.2,88,67,31.3
91,20,132605.0,FELONY ASSAULT,80,52,-35.0,34,29,17.2
107,23,31759.5,FELONY ASSAULT,330,318,-3.6,207,185,11.9
123,25,31759.5,FELONY ASSAULT,329,317,-3.6,197,171,15.2


In [233]:
len(df_felonyassault[df_felonyassault['%_Chg'] > 5.8])

37

In [234]:
df_felonyassault[df_felonyassault['%_Chg'] > 5.8].household_median_income.median()

58872.0

In [235]:
len(df_felonyassault[df_felonyassault['%_Chg'] < 5.8])

40

In [236]:
df_felonyassault[df_felonyassault['%_Chg'] < 5.8].household_median_income.median()

64311.75

### burglary

In [237]:
df_burglary = df_MF4[df_MF4.CRIME.str.strip() == "BURGLARY"]

In [238]:
((df_burglary['2021_YTD'].sum() - df_burglary['2020_YTD'].sum()) / df_burglary['2020_YTD'].sum())*100 

-23.707685269899358

In [239]:
df_burglary[df_burglary['%_Chg']> -23.7]

,PCT,household_median_income,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
36,9,63717.0,BURGLARY,123,270,119.5,176,160,10.0
60,14,95301.0,BURGLARY,178,353,98.3,236,201,17.4
140,28,46263.0,BURGLARY,90,147,63.3,85,89,-4.5
148,30,53660.0,BURGLARY,69,103,49.3,77,58,32.8
156,32,46263.0,BURGLARY,110,128,16.4,98,61,60.7
204,43,37503.0,BURGLARY,140,244,74.3,136,134,1.5
212,44,29579.0,BURGLARY,173,253,46.2,151,155,-2.6
220,45,56291.5,BURGLARY,99,101,2.0,57,69,-17.4
236,47,58393.0,BURGLARY,245,241,-1.6,148,137,8.0
244,48,29136.0,BURGLARY,150,237,58.0,118,149,-20.8


In [240]:
len(df_burglary[df_burglary['%_Chg']> -23.7])

37

In [241]:
df_burglary[df_burglary['%_Chg']> -23.7].household_median_income.median()

60569.0

In [242]:
len(df_burglary[df_burglary['%_Chg'] < -23.7])

40

In [243]:
df_burglary[df_burglary['%_Chg'] < -23.7].household_median_income.median()

61626.0

### grand larceny

In [244]:
df_grandlarceny = df_MF4[df_MF4.CRIME.str.strip() == "GRAND LARCENY"]
# requires that the property value exceed $50,000

In [245]:
((df_grandlarceny['2021_YTD'].sum() - df_grandlarceny['2020_YTD'].sum()) / df_grandlarceny['2020_YTD'].sum())*100 

1.010048689526572

In [246]:
len(df_grandlarceny[df_grandlarceny['%_Chg'] > 1])

40

In [247]:
df_grandlarceny[df_grandlarceny['%_Chg']> 1].household_median_income.median()

56863.25

In [248]:
len(df_grandlarceny[df_grandlarceny['%_Chg'] < 1])

37

In [249]:
df_grandlarceny[df_grandlarceny['%_Chg'] < 1].household_median_income.median()

67642.0

## motor vehicle heft

In [250]:
df_motorvehicle = df_MF4[df_MF4.CRIME.str.strip() == "GRAND LARCENY OF MOTOR VEHICLE"]

In [251]:
((df_motorvehicle['2021_YTD'].sum() - df_motorvehicle['2020_YTD'].sum()) / df_motorvehicle['2020_YTD'].sum())*100 

22.79981760145919

In [252]:
df_motorvehicle[df_motorvehicle['%_Chg'] > 22.8]

,PCT,household_median_income,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
6,1,186231.0,GRAND LARCENY OF MOTOR VEHICLE,19,31,63.2,28,17,64.7
14,5,186231.0,GRAND LARCENY OF MOTOR VEHICLE,16,24,50.0,20,12,66.7
30,7,63717.0,GRAND LARCENY OF MOTOR VEHICLE,18,55,205.6,41,25,64.0
38,9,63717.0,GRAND LARCENY OF MOTOR VEHICLE,37,61,64.9,59,24,145.8
46,10,113138.0,GRAND LARCENY OF MOTOR VEHICLE,23,49,113.0,31,23,34.8
70,17,132016.5,GRAND LARCENY OF MOTOR VEHICLE,16,48,200.0,31,21,47.6
86,19,122569.0,GRAND LARCENY OF MOTOR VEHICLE,65,121,86.2,90,43,109.3
94,20,132605.0,GRAND LARCENY OF MOTOR VEHICLE,38,44,15.8,53,10,430.0
118,24,132605.0,GRAND LARCENY OF MOTOR VEHICLE,39,77,97.4,48,30,60.0
126,25,31759.5,GRAND LARCENY OF MOTOR VEHICLE,21,51,142.9,42,20,110.0


In [253]:
len(df_motorvehicle[df_motorvehicle['%_Chg'] > 22.8])

33

In [254]:
df_motorvehicle[df_motorvehicle['%_Chg'] > 22.8].household_median_income.median()

57435.0

In [255]:
len(df_motorvehicle[df_motorvehicle['%_Chg'] < 22.8])

44

In [256]:
df_motorvehicle[df_motorvehicle['%_Chg'] <22.8].household_median_income.median()

66274.25

### total seven major crimes

In [257]:
df_total = df_MF4[df_MF4.CRIME.str.strip() == "TOTAL SEVEN MAJOR FELONY OFFENSES"]

In [258]:
((df_total['2021_YTD'].sum() - df_total['2020_YTD'].sum()) / df_total['2020_YTD'].sum())*100 

-0.3873528345450035

In [259]:
df_total[df_total['%_Chg'] > -0.4]

,PCT,household_median_income,CRIME,2019,2020,%_chg_2019-2020,2021_YTD,2020_YTD,%_Chg
31,7,63717.0,TOTAL SEVEN MAJOR FELONY OFFENSES,962,941,-2.2,567,502,12.9
39,9,63717.0,TOTAL SEVEN MAJOR FELONY OFFENSES,1234,1202,-2.6,803,642,25.1
63,14,95301.0,TOTAL SEVEN MAJOR FELONY OFFENSES,2552,1990,-22.0,1462,1059,38.1
95,20,132605.0,TOTAL SEVEN MAJOR FELONY OFFENSES,900,835,-7.2,559,438,27.6
111,23,31759.5,TOTAL SEVEN MAJOR FELONY OFFENSES,917,1078,17.6,616,609,1.1
119,24,132605.0,TOTAL SEVEN MAJOR FELONY OFFENSES,1017,922,-9.3,524,497,5.4
143,28,46263.0,TOTAL SEVEN MAJOR FELONY OFFENSES,880,883,0.3,552,478,15.5
151,30,53660.0,TOTAL SEVEN MAJOR FELONY OFFENSES,777,774,-0.4,455,432,5.3
159,32,46263.0,TOTAL SEVEN MAJOR FELONY OFFENSES,1088,1100,1.1,696,613,13.5
167,33,53660.0,TOTAL SEVEN MAJOR FELONY OFFENSES,779,959,23.1,613,527,16.3


In [260]:
len(df_total[df_total['%_Chg'] > -0.4])

34

In [261]:
df_total[df_total['%_Chg'] > -0.4].household_median_income.median()

57914.0

In [262]:
len(df_total[df_total['%_Chg'] < -0.4])

43

In [263]:
df_total[df_total['%_Chg'] < -0.4].household_median_income.median()

64906.5

In [264]:
len(df_total[df_total['%_Chg'] < 0])

43

In [265]:
df_total[df_total['%_Chg'] < 0].household_median_income.median()

64906.5

In [266]:
df_total[df_total['%_Chg'] > 0].household_median_income.median()

57914.0

In [267]:
len(df_total[df_total['%_Chg'] > 0])

34

In [268]:
len(df_total[df_total['%_Chg'] < 0])

43

In [269]:
len(df_19_20[df_19_20['%_chg_2019-2020'] < 0])

NameError: name 'df_19_20' is not defined

In [ ]:
len(df_19_20[df_19_20['%_chg_2019-2020'] > 0])

In [270]:
MF4.head(1)

NameError: name 'MF4' is not defined

# Step 6: Violent crimes by precinct (murder, rape, robbery, felon assault)
### violent crimes rose abt 3.4% YoY between Jan.-Aug. 2 2021; dropped in 2020, rose 2.6% in 2019
#### violent crimes rose > 0 in 39 precicnts; top 5: precints 14, 22, 17, 33, 63  led by rises in ROBBERY & FELONY ASSAULT; both have been on a downward trend in the past five years 2016-2020 

In [ ]:
Violent_Crimes = df_MF4[(df_MF4['CRIME'] == "MURDER & NON NEGL. MANSLAUGHTER") | (df_MF4['CRIME'] == "RAPE") | (df_MF4['CRIME'] == "ROBBERY") | (df_MF4['CRIME'] == "FELONY ASSAULT")]

In [ ]:
Violent_Crimes = Violent_Crimes[['PCT', 'CRIME', '2021_YTD', '2020_YTD', '%_Chg','household_median_income']]

In [ ]:
(Violent_Crimes['2021_YTD'].sum() - Violent_Crimes['2020_YTD'].sum()) / Violent_Crimes['2020_YTD'].sum() * 100

In [ ]:
Violent_Crimes.head(1)

In [ ]:
# grouping by PCT and getting the sum of 2021_YTD
ViolentCrimes2021 = Violent_Crimes.groupby("PCT")['2021_YTD'].sum()

In [ ]:
ViolentCrimes2021 = ViolentCrimes2021.to_frame()
ViolentCrimes2021.reset_index(level=0, inplace=True)
#ViolentCrimes2021 = ViolentCrimes2021.rename(columns={0: "NumOfHomicides"})

In [ ]:
ViolentCrimes2021 = ViolentCrimes2021.sort_values(by="PCT", ascending = True)

In [ ]:
ViolentCrimes2020 = Violent_Crimes.groupby("PCT")['2020_YTD'].sum()

In [ ]:
ViolentCrimes2020 = ViolentCrimes2020.to_frame()
ViolentCrimes2020.reset_index(level=0, inplace=True)
#ViolentCrimes2020 = ViolentCrimes2020.rename(columns={0: "NumOfHomicides"})

In [ ]:
ViolentCrimes2020 = ViolentCrimes2020.sort_values(by="PCT", ascending = True)

In [ ]:
ViolentCrimes_Precinct = pd.merge(ViolentCrimes2020, ViolentCrimes2021, on='PCT', how='outer')

In [ ]:
ViolentCrimes_Precinct['%chng'] = (ViolentCrimes_Precinct['2021_YTD'] - ViolentCrimes_Precinct['2020_YTD']) / ViolentCrimes_Precinct['2020_YTD'] * 100

In [ ]:
ViolentCrimes_Precinct['%chng'] = round(ViolentCrimes_Precinct['%chng'], 1)

In [ ]:
len(ViolentCrimes_Precinct[ViolentCrimes_Precinct['%chng'] > 0])

In [ ]:
# IMPORTANT
ViolentCrimes_Precinct.sort_values(by="%chng", ascending=False)

In [ ]:
# spot checking
Violent_Crimes[Violent_Crimes.PCT == 66]

In [ ]:
# IMPORTANT
ViolentCrimes_Precinct.sort_values(by="%chng", ascending=True)

In [ ]:
# # murder
# Violent_Crimes_murder = df_MF4[(df_MF4['CRIME'] == "MURDER & NON NEGL. MANSLAUGHTER")]

In [ ]:
# Violent_Crimes_murder.to_csv("Violent_Crimes_murder.csv", index=False)

In [ ]:
# # rape
# Violent_Crimes_rape = df_MF4[(df_MF4['CRIME'] == "RAPE")]
# Violent_Crimes_rape.to_csv("Violent_Crimes_rape.csv", index=False)

In [ ]:
# # robbery
# Violent_Crimes_robbery = df_MF4[(df_MF4['CRIME'] == "ROBBERY")]
# Violent_Crimes_robbery.to_csv("Violent_Crimes_robbery.csv", index=False)

In [ ]:
# # felony assault
# Violent_Crimes_felonyassault = df_MF4[(df_MF4['CRIME'] == "FELONY ASSAULT")]
# Violent_Crimes_felonyassault.to_csv("Violent_Crimes_felonyassault.csv", index=False)

In [ ]:
# Violent_Crimes.to_excel("Violent_Crimes.xlsx",index=False)

In [ ]:
# Violent_Crimes_new = pd.read_excel('Violent_Crimes_new.xlsx')

In [ ]:
# Violent_Crimes_new

In [ ]:
df_MF2

In [ ]:
df_MF2['OFFENSE'] = df_MF2.index

In [ ]:
df_MF2

In [ ]:
# murder, rape, robbery, felon assault
annual_ViolentCrimes = df_MF2[(df_MF2.OFFENSE=="MURDER & NON-NEGL. MANSLAUGHTER")|\
                               (df_MF2.OFFENSE=="RAPE")|(df_MF2.OFFENSE=="ROBBERY")|\
                              (df_MF2.OFFENSE=="FELONY ASSAULT")]

In [ ]:
# IMPORTANT
annual_ViolentCrimes

In [ ]:
(annual_ViolentCrimes[2020.0].sum() - annual_ViolentCrimes[2019.0].sum()) / annual_ViolentCrimes[2019.0].sum() * 100

In [ ]:
(annual_ViolentCrimes[2019.0].sum() - annual_ViolentCrimes[2018.0].sum()) / annual_ViolentCrimes[2018.0].sum() * 100

### comparing overall crimes with Google mobility data

In [ ]:
# os.chdir("..")

In [ ]:
# Google mobility: https://www.google.com/covid19/mobility/
# Data updated Aug.8

df_mobility = pd.read_csv("2021_US_Region_Mobility_Report.csv")

In [ ]:
# filtering for: New York County (Manhattan), Kings County (Brooklyn), 
# Bronx County (The Bronx), Richmond County (Staten Island) and Queens County (Queens).
df_NYmobility =  df_mobility[(df_mobility.sub_region_2 == "Bronx County") |\
                             (df_mobility.sub_region_2 == "New York County") | \
            (df_mobility.sub_region_2 == "Kings County") | (df_mobility.sub_region_2 == "Richmond County")| \
            (df_mobility.sub_region_2 == "Queens County")]

In [ ]:
df_NYmobility.sub_region_2=df_NYmobility.sub_region_2.replace({'Bronx County':'Bronx',\
                                                    'New York County':'Manhattan',\
                                                    'Kings County':'Brooklyn',\
                                                    'Richmond County':'Staten Island',\
                                                    'Queens County':'Queens'})

In [ ]:
df_NYmobility = df_NYmobility[df_NYmobility.sub_region_1 == "New York"]

In [ ]:
df_NYmobility = df_NYmobility.reset_index(drop=True)

In [ ]:
df_NYmobility.retail_and_recreation_percent_change_from_baseline =\
df_NYmobility.retail_and_recreation_percent_change_from_baseline.astype(int)

In [ ]:
df_NYmobility.grocery_and_pharmacy_percent_change_from_baseline =\
df_NYmobility.grocery_and_pharmacy_percent_change_from_baseline.astype(int)

In [ ]:
df_NYmobility.parks_percent_change_from_baseline =\
df_NYmobility.parks_percent_change_from_baseline.astype(int)

In [ ]:
df_NYmobility.transit_stations_percent_change_from_baseline =\
df_NYmobility.transit_stations_percent_change_from_baseline.astype(int)

In [ ]:
df_NYmobility.workplaces_percent_change_from_baseline =\
df_NYmobility.workplaces_percent_change_from_baseline.astype(int)

In [ ]:
df_NYmobility.residential_percent_change_from_baseline =\
df_NYmobility.residential_percent_change_from_baseline.astype(int)

In [ ]:
df_NYmobility =df_NYmobility.rename(columns={"sub_region_2": "Borough",\
                              "retail_and_recreation_percent_change_from_baseline": "Retail, Recreation % Change",\
                             "parks_percent_change_from_baseline":"Parks % Change",\
                             "workplaces_percent_change_from_baseline":"Workplaces % Change",\
                             "residential_percent_change_from_baseline":"Residential % Change",\
                             "transit_stations_percent_change_from_baseline": "Transit Station % Change"})

In [ ]:
df_NYmobility = df_NYmobility.rename(columns={"date": "Date"})

In [ ]:
df_NYmobility.head(1)

In [ ]:
# IMPORTANT
from vega_datasets import data
import altair as alt

alt.Chart(df_NYmobility).mark_circle().encode(
    alt.X("Date:T"),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='Borough:N'
).properties(
    width=250,
    height=270,
).repeat(
    row=['Retail, Recreation % Change',\
         'Parks % Change', 'Transit Station % Change',\
        'Residential % Change'],\
    column=['Date'],
    title = "Mobility trends in NYC by borough, Jan.-Aug.6 2021"
)

## NYPD historic data

In [ ]:
#os.chdir("..")

In [ ]:
# nypd historic complaint data until March 30
# https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i
df_complaint = pd.read_csv("NYPD_Complaint_Data_Historic.csv")

In [ ]:
df_complaint.shape

In [ ]:
# df_Robbery_FelonyAssault = df_complaint[(df_complaint.OFNS_DESC == "ROBBERY") |\
#                                         (df_complaint.OFNS_DESC == "FELONY ASSAULT")]

In [ ]:
# df_Robbery_FelonyAssault = df_Robbery_FelonyAssault.reset_index(drop=True)

In [ ]:
df_complaint=df_complaint.rename(columns={"BORO_NM":"Borough",\
                                                                 "OFNS_DESC":"Crime"})

In [ ]:
df_complaint=df_complaint.rename(columns={"CMPLNT_FR_DT":"Date"})

In [ ]:
df_complaint.Date = df_complaint.Date.astype(str)

In [ ]:
import re
df_complaint['Month']=df_complaint.Date.apply(lambda txt: re.findall("^\d\d\D", txt))

In [ ]:
df_complaint.Month = df_complaint.Month.astype(str)

In [ ]:
df_complaint.head(2)

In [ ]:
df_complaint.Month = df_complaint.Month.str.replace("[","")
df_complaint.Month = df_complaint.Month.str.replace("]","")
df_complaint.Month = df_complaint.Month.str.replace("'","")
df_complaint.Month = df_complaint.Month.str.replace("/","")

In [ ]:
df_complaint['Year']=df_complaint.Date.apply(lambda txt: re.findall("\d\d\d\d$", txt))

In [ ]:
df_complaint.Year = df_complaint.Year.astype(str)

In [ ]:
df_complaint.Year = df_complaint.Year.str.replace("[","")
df_complaint.Year = df_complaint.Year.str.replace("]","")
df_complaint.Year = df_complaint.Year.str.replace("'","")

In [ ]:
df_complaint.head(2)

## NYPD complaint data second half of 2020 & 2019
### July 19

In [ ]:
df_complaint2020 = df_complaint[df_complaint.Year == '2020']

In [ ]:
# Some 300 homicide reports in 2020
df_complaint2020.value_counts("Crime").head(10)

In [ ]:
df_complaint2020.head(1)

In [ ]:
df_complaint2020_h2 = df_complaint2020[(df_complaint2020.Month == '06') | (df_complaint2020.Month == '07') |\
                 (df_complaint2020.Month == '08') | (df_complaint2020.Month == '09') |\
                (df_complaint2020.Month == '10') | (df_complaint2020.Month == '11')|\
                (df_complaint2020.Month == '12')]

In [ ]:
df_complaint2020_new = df_complaint2020_h2.groupby("Month").count()

In [ ]:
df_complaint2020_new

In [ ]:
df_complaint2020_new['Month'] = df_complaint2020_new.index

In [ ]:
col_list = ['CMPLNT_NUM','Month']
df_complaint2020_new = df_complaint2020_new[col_list]
df_complaint2020_new

In [ ]:
df_complaint2020_new = df_complaint2020_new.rename(columns={"CMPLNT_NUM":"Crimes_2020"})

In [ ]:
# combining 'df_Precincts_Income_Final_update' with df_complaint202

In [ ]:
df_complaint2020.ADDR_PCT_CD = df_complaint2020.ADDR_PCT_CD.astype(int)

In [ ]:
complaint2020_gb = df_complaint2020.rename(columns={"ADDR_PCT_CD":"Precinct"})

In [ ]:
col_list = ['CMPLNT_NUM', 'Precinct']
complaint2020_gb = complaint2020_gb[col_list]

In [ ]:
complaint2020_gb = complaint2020_gb.rename(columns={"CMPLNT_NUM":"totalComplaints"})

In [ ]:
complaint2020_gb  = complaint2020_gb.groupby("Precinct").count()

In [ ]:
complaint2020_gb

In [ ]:
df_Precincts_Income_Final_update = pd.read_csv("df_Precincts_Income_Final_update.csv")

In [ ]:
df_Precincts_Income_Final_update.shape

In [ ]:
testing = pd.merge(complaint2020_gb, df_Precincts_Income_Final_update, how="left", on=["Precinct"])

In [ ]:
testingUpper = testing[testing.incomeTier == "Upper"]

In [ ]:
testingUpper.shape

In [ ]:
testing = testing[testing.Precinct != 22]

In [ ]:
testingLower = testing[testing.incomeTier == "Lower"]

In [ ]:
testing.head(1)

In [ ]:
testing = testing.sort_values("totalComplaints", ascending=False)

In [ ]:
import altair as alt
from vega_datasets import data

top20 = alt.Chart(testing.head(10)).mark_bar(opacity=0.8).encode(
    alt.X("totalComplaints:Q", title="Total Number of Complaints", scale=alt.Scale(domain=[0, 13000])),
    alt.Y("Precinct:N", sort='-x'),
    color=alt.Color('incomeTier', scale=alt.Scale(
            domain=['Lower', 'Upper'],
            range=['purple', 'yellow']))).properties(
title="Top 10 precincts with greatest police complaints in 2020")

In [ ]:
import altair as alt
from vega_datasets import data

bottom20 = alt.Chart(testing.tail(10)).mark_bar(opacity=0.8).encode(
    alt.X("totalComplaints:Q", title="Total Number of Complaints", scale=alt.Scale(domain=[0, 13000])),
    alt.Y("Precinct:N", sort='-x'),
    color=alt.Color('incomeTier', scale=alt.Scale(
            domain=['Lower', 'Upper'],
            range=['purple', 'yellow']))).properties(
title="Bottom 20 precincts with fewest police complaints in 2020")

In [ ]:
top20 | bottom20

In [ ]:
# reading NYPD precinct map

In [ ]:
nypd = gpd.read_file('https://opendata.arcgis.com/datasets/c35786feb0ac4d1b964f41f874f151c1_0.geojson')

In [ ]:
# ask Juan and Michael

In [ ]:
nypd.head(1)

In [ ]:
testing.head(1)

In [ ]:
df_nypd = pd.merge(testing, nypd , on="Precinct")

In [ ]:
nypdMap = alt.Chart(df_nypd).mark_geoshape().encode(color=alt.Color('totalComplaints:Q',
                        legend=alt.Legend(title='Income Tier')
                        ), 
                        tooltip=[alt.Tooltip('Precinct')]
).properties(
    title='Avg. Bridge Rating by County',
    width=500,
    height=600
)

# .project("albersUsa")

In [ ]:
nypdMap

In [ ]:
df_complaint2019 = df_complaint[df_complaint.Year == '2019']

In [ ]:
df_complaint2019_h2 = df_complaint2019[(df_complaint2019.Month == '06') | (df_complaint2019.Month == '07') |\
                 (df_complaint2019.Month == '08') | (df_complaint2019.Month == '09') |\
                (df_complaint2019.Month == '10') | (df_complaint2019.Month == '11')|\
                (df_complaint2019.Month == '12')]

In [ ]:
df_complaint2019_new = df_complaint2019_h2.groupby("Month").count()

In [ ]:
df_complaint2019_new['Month'] = df_complaint2019_new.index

In [ ]:
col_list = ['CMPLNT_NUM','Month']
df_complaint2019_new = df_complaint2019_new[col_list]
df_complaint2019_new

In [ ]:
df_complaint2019_new = df_complaint2019_new.rename(columns={"CMPLNT_NUM":"Crimes_2019"})

In [ ]:
df_complaint2019_new = df_complaint2019_new.rename(columns={"Month":"month"})

In [ ]:
df_complaint2020_new = df_complaint2020_new.rename(columns={"Month":"month"})

In [ ]:
df_complaint_final = pd.merge(df_complaint2019_new, df_complaint2020_new , on="month")

In [ ]:
df_complaint_final

In [ ]:
df_complaint2018 = df_complaint[df_complaint.Year == '2018']

In [ ]:
df_complaint2018_h2 = df_complaint2018[(df_complaint2018.Month == '06') | (df_complaint2018.Month == '07') |\
                 (df_complaint2018.Month == '08') | (df_complaint2018.Month == '09') |\
                (df_complaint2018.Month == '10') | (df_complaint2018.Month == '11')|\
                (df_complaint2018.Month == '12')]

In [ ]:
df_complaint2018_new = df_complaint2018_h2.groupby("Month").count()

In [ ]:
df_complaint2018_new['Month'] = df_complaint2018_new.index

In [ ]:
col_list = ['CMPLNT_NUM','Month']
df_complaint2018_new = df_complaint2018_new[col_list]
df_complaint2018_new

In [ ]:
df_complaint2018_new = df_complaint2018_new.rename(columns={"Month":"month"})

In [ ]:
df_complaint2018_new = df_complaint2018_new.rename(columns={"CMPLNT_NUM":"Crimes_2018"})

In [ ]:
df_complaint_final = pd.merge(df_complaint2018_new, df_complaint_final , on="month")

In [ ]:
df_complaint_final = df_complaint_final[df_complaint_final.month != "06"]

In [ ]:
# IMPORTANT
import matplotlib.pyplot as plt

plt.plot(df_complaint_final['Crimes_2018'], label = '2018', linewidth=2, marker='o', markersize=8)
plt.plot(df_complaint_final['Crimes_2019'], label = '2019', linewidth=2, marker='o', markersize=8)
plt.plot(df_complaint_final['Crimes_2020'], label = '2020', linewidth=5, marker='o', markersize=12)

# add legend
plt.legend()

#add axis labels and a title
plt.ylabel('Total number of complaints', fontsize=14)
plt.xlabel('Month', fontsize=14)
plt.title('Complaints filed to the NYPD between July-Dec.', fontsize=16)

# customizing x-axis
values = ['July', 'Aug.', 'Sept.', 'Oct.','Nov.','Dec.'] 
x=[1,2,3,4,5,6]
plt.xticks(x,values)
    
plt.show()

In [ ]:
# finding complaints filed in 2020 by income

In [ ]:
df_complaint2020.ADDR_PCT_CD = df_complaint2020.ADDR_PCT_CD.astype(int)

In [ ]:
df_complaint2020.head(1)

In [ ]:
df_complaint2020_update = df_complaint2020.rename(columns={"DDR_PCT_CD": "Precinct"})

In [ ]:
df_Precincts_Income_Final_update.head(5)

In [ ]:
col_list = ["Precinct", "median_of_household_incomes"]
df_Precincts_Income_Final_update = df_Precincts_Income_Final[col_list]

In [ ]:
df_Precincts_Income_Final_update['incomeTier'] = df_Precincts_Income_Final_update.median_of_household_incomes > 68920 

In [ ]:
df_Precincts_Income_Final_update['incomeTier'] = df_Precincts_Income_Final_update['incomeTier'].astype(str)

In [ ]:
df_Precincts_Income_Final_update['incomeTier'] = df_Precincts_Income_Final_update['incomeTier'].str.replace('True', 'Upper')

In [ ]:
df_Precincts_Income_Final_update['incomeTier'] =df_Precincts_Income_Final_update['incomeTier'].str.replace('False', 'Lower')

In [ ]:
df_Precincts_Income_Final_update.to_csv("df_Precincts_Income_Final_update.csv")

## suspect, victime by age
### July 31 2021

In [ ]:
# df_complaint2020, df_complaint2019, df_complaint2018

In [ ]:
df_complaint2020_suspect = df_complaint2020[df_complaint2020.SUSP_AGE_GROUP != "UNKNOWN"]

In [ ]:
df_complaint2020_suspect.SUSP_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
df_complaint2020_victim = df_complaint2020[df_complaint2020.VIC_AGE_GROUP != "UNKNOWN"]

In [ ]:
df_complaint2020_victim.VIC_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
# violent crimes: ROBBERY, RAPE, ASSAULT 3 & RELATED OFFENSES, MURDER & NON-NEGL. MANSLAUGHTER,
df_complaint2020_violent = df_complaint2020[(df_complaint2020.Crime == "ROBBERY") | (df_complaint2020.Crime == "RAPE")|\
                                             (df_complaint2020.Crime == "ASSAULT 3 & RELATED OFFENSES") | \
                                             (df_complaint2020.Crime == "MURDER & NON-NEGL. MANSLAUGHTER")]

In [ ]:
df_complaint2020_violent_suspect = df_complaint2020_violent[df_complaint2020_violent.SUSP_AGE_GROUP != "UNKNOWN"]

In [ ]:
df_complaint2020_violent_suspect.SUSP_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
df_complaint2019_suspect = df_complaint2019[df_complaint2019.SUSP_AGE_GROUP != "UNKNOWN"]

In [ ]:
df_complaint2019_suspect.SUSP_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
df_complaint2019_victim = df_complaint2019[df_complaint2019.VIC_AGE_GROUP != "UNKNOWN"]

In [ ]:
df_complaint2019_victim.VIC_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
df_complaint2018_suspect = df_complaint2018[df_complaint2018.SUSP_AGE_GROUP != "UNKNOWN"]

In [ ]:
df_complaint2018_suspect.SUSP_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
df_complaint2018_victim = df_complaint2018[df_complaint2018.VIC_AGE_GROUP != "UNKNOWN"]

In [ ]:
df_complaint2021_victim = df_complaint2021[df_complaint2021.VIC_AGE_GROUP != "UNKNOWN"]

In [ ]:
df_complaint2021_victim.VIC_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
df_complaint2018_victim.VIC_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
df_complaint2021_suspect = df_complaint2021[df_complaint2021.SUSP_AGE_GROUP != "UNKNOWN"]

In [ ]:
df_complaint2021_suspect.SUSP_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
# violent crimes: ROBBERY, RAPE, ASSAULT 3 & RELATED OFFENSES, MURDER & NON-NEGL. MANSLAUGHTER,
df_complaint2019_violent = df_complaint2019[(df_complaint2019.Crime == "ROBBERY") | (df_complaint2019.Crime == "RAPE")|\
                                             (df_complaint2019.Crime == "ASSAULT 3 & RELATED OFFENSES") | \
                                             (df_complaint2019.Crime == "MURDER & NON-NEGL. MANSLAUGHTER")]

df_complaint2019_violent_suspect = df_complaint2019_violent[df_complaint2019_violent.SUSP_AGE_GROUP != "UNKNOWN"]

df_complaint2019_violent_suspect.SUSP_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
# violent crimes: ROBBERY, RAPE, ASSAULT 3 & RELATED OFFENSES, MURDER & NON-NEGL. MANSLAUGHTER,
df_complaint2018_violent = df_complaint2018[(df_complaint2018.Crime == "ROBBERY") | (df_complaint2018.Crime == "RAPE")|\
                                             (df_complaint2018.Crime == "ASSAULT 3 & RELATED OFFENSES") | \
                                             (df_complaint2018.Crime == "MURDER & NON-NEGL. MANSLAUGHTER")]

df_complaint2018_violent_suspect = df_complaint2018_violent[df_complaint2018_violent.SUSP_AGE_GROUP != "UNKNOWN"]

df_complaint2018_violent_suspect.SUSP_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
df_complaint2021.head(1)

In [ ]:
# violent crimes: ROBBERY, RAPE, ASSAULT 3 & RELATED OFFENSES, MURDER & NON-NEGL. MANSLAUGHTER,
df_complaint2021_violent = df_complaint2021[(df_complaint2021.OFNS_DESC == "ROBBERY") | (df_complaint2021.OFNS_DESC == "RAPE")|\
                                             (df_complaint2021.OFNS_DESC == "ASSAULT 3 & RELATED OFFENSES") | \
                                             (df_complaint2021.OFNS_DESC == "MURDER & NON-NEGL. MANSLAUGHTER")]

df_complaint2021_violent_suspect = df_complaint2021_violent[df_complaint2021_violent.SUSP_AGE_GROUP != "UNKNOWN"]

df_complaint2021_violent_suspect.SUSP_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
# violent crimes: ROBBERY, RAPE, ASSAULT 3 & RELATED OFFENSES, MURDER & NON-NEGL. MANSLAUGHTER,
df_complaint2021_violent = df_complaint2021[(df_complaint2021.OFNS_DESC == "ROBBERY") | (df_complaint2021.OFNS_DESC == "RAPE")|\
                                             (df_complaint2021.OFNS_DESC == "ASSAULT 3 & RELATED OFFENSES") | \
                                             (df_complaint2021.OFNS_DESC == "MURDER & NON-NEGL. MANSLAUGHTER")]

df_complaint2021_violent_suspect = df_complaint2021_violent[df_complaint2021_violent.VIC_AGE_GROUP != "UNKNOWN"]

df_complaint2021_violent_suspect.VIC_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
df_complaint2020.head(1)

In [ ]:
# violent crimes: ROBBERY, RAPE, ASSAULT 3 & RELATED OFFENSES, MURDER & NON-NEGL. MANSLAUGHTER,
df_complaint2020_violent = df_complaint2020[(df_complaint2020.Crime == "ROBBERY") | (df_complaint2020.Crime == "RAPE")|\
                                             (df_complaint2020.Crime == "ASSAULT 3 & RELATED OFFENSES") | \
                                             (df_complaint2020.Crime == "MURDER & NON-NEGL. MANSLAUGHTER")]

df_complaint2020_violent_suspect = df_complaint2020_violent[df_complaint2020_violent.VIC_AGE_GROUP != "UNKNOWN"]

df_complaint2020_violent_suspect.VIC_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
# violent crimes: ROBBERY, RAPE, ASSAULT 3 & RELATED OFFENSES, MURDER & NON-NEGL. MANSLAUGHTER,
df_complaint2019_violent = df_complaint2019[(df_complaint2019.Crime == "ROBBERY") | (df_complaint2019.Crime == "RAPE")|\
                                             (df_complaint2019.Crime == "ASSAULT 3 & RELATED OFFENSES") | \
                                             (df_complaint2019.Crime == "MURDER & NON-NEGL. MANSLAUGHTER")]

df_complaint2019_violent_suspect = df_complaint2019_violent[df_complaint2019_violent.VIC_AGE_GROUP != "UNKNOWN"]

df_complaint2019_violent_suspect.VIC_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
# violent crimes: ROBBERY, RAPE, ASSAULT 3 & RELATED OFFENSES, MURDER & NON-NEGL. MANSLAUGHTER,
df_complaint2018_violent = df_complaint2018[(df_complaint2018.Crime == "ROBBERY") | (df_complaint2018.Crime == "RAPE")|\
                                             (df_complaint2018.Crime == "ASSAULT 3 & RELATED OFFENSES") | \
                                             (df_complaint2018.Crime == "MURDER & NON-NEGL. MANSLAUGHTER")]

df_complaint2018_violent_suspect = df_complaint2018_violent[df_complaint2018_violent.VIC_AGE_GROUP != "UNKNOWN"]

df_complaint2018_violent_suspect.VIC_AGE_GROUP.value_counts(normalize=True) * 100

In [ ]:
# suspectByAgeGroup.xlsx
# victimByAgeGroup.xlsx

In [ ]:
violent_susp_ageGroup = pd.read_excel("violent_suspectByAgeGroup.xlsx")

In [ ]:
violent_susp_ageGroup

In [ ]:
# IMPORTANT

import altair as alt
from vega_datasets import data

chart = alt.Chart(violent_vic_ageGroup, title="Victims' age group 2018-2021").mark_bar(opacity=0.8).encode(
    alt.X('Percentage:Q', title="Percentage of Victims", scale=alt.Scale(domain=[0, 60])),
    y='Year:N',
    color='Age Group:N',
     row='Age Group:N'
).properties(width=500)

chart.configure_title(fontSize=20, offset=5, orient='top', anchor='middle')

In [ ]:
chart2 = alt.Chart(violent_susp_ageGroup, title="Suspects' age group 2018-2021").mark_bar(opacity=0.8).encode(
    alt.X('Percentage:Q', title="Percentage of Suspects"),
    y='Year:N',
    color='Age Group:N',
     row='Age Group:N'
).properties(width=500)

chart2.configure_title(fontSize=20, offset=5, orient='top', anchor='middle')

In [ ]:
chart | chart2

In [ ]:
vic_ageGroup = pd.read_excel("victimByAgeGroup.xlsx")

In [ ]:
vic_ageGroup = vic_ageGroup.set_index('victim_ageGroup')

In [ ]:
vic_ageGroup_violent = vic_ageGroup.drop(columns=['victim_ageGroup_2021', 'victim_ageGroup_2020',\
                                                 'victim_ageGroup_2019', 'victim_ageGroup_2018'])

In [ ]:
vic_ageGroup_violent

## complaint 2021

In [ ]:
# updated quarterly
# https://www1.nyc.gov/site/nypd/stats/crime-statistics/citywide-crime-stats.page

In [ ]:
df_complaint2021 = pd.read_csv("NYPD_Complaint_Data_YTD_FINAL.csv")

In [ ]:
df_complaint2021.shape

In [ ]:
df_complaint2021 = df_complaint2021.rename(columns={"CMPLNT_FR_DT":"Date"})

In [ ]:
df_complaint2021.head(1)

In [ ]:
## making complaint line graph by month to compare with Google mobility data

In [ ]:
df_complaint2021['Year']=df_complaint2021.Date.apply(lambda txt: re.findall("\d\d\d\d$", txt))

In [ ]:
df_complaint2021['Month']=df_complaint2021.Date.apply(lambda txt: re.findall("^\d\d", txt))

In [ ]:
df_complaint2021.Year = df_complaint2021.Year.astype(str)
df_complaint2021.Year = df_complaint2021.Year.str.replace("[","")
df_complaint2021.Year = df_complaint2021.Year.str.replace("]","")

In [ ]:
df_complaint2021.Year = df_complaint2021.Year.str.replace("'","")

In [ ]:
df_complaint2021.Month = df_complaint2021.Month.astype(str)
df_complaint2021.Month = df_complaint2021.Month.str.replace("[","")
df_complaint2021.Month = df_complaint2021.Month.str.replace("]","")

In [ ]:
df_complaint2021.Month = df_complaint2021.Month.str.replace("'","")

In [ ]:
df_complaint2021 = df_complaint2021[df_complaint2021.Year=="2021"]

In [ ]:
df_complaint2021_groupby = df_complaint2021.groupby("Month").count()

In [ ]:
df_complaint2021_groupby 

In [ ]:
col_list = ["CMPLNT_NUM"]
df_complaint2021_groupby = df_complaint2021_groupby[col_list]

In [ ]:
df_complaint2021_groupby['month'] = df_complaint2021_groupby.index

In [ ]:
df_complaint2021_groupby = df_complaint2021_groupby.rename(columns={"CMPLNT_NUM": "Total Number of Complaints"})

In [ ]:
df_complaint2021_groupby = df_complaint2021_groupby.rename(columns={"Total Number of Complaints": "TotalComplaints2021"})

In [ ]:
df_complaint2020_h1 = df_complaint2020[(df_complaint2020.Month == '01') | (df_complaint2020.Month == '02') |\
                 (df_complaint2020.Month == '03') | (df_complaint2020.Month == '04') |\
                (df_complaint2020.Month == '05') | (df_complaint2020.Month == '06')]

In [ ]:
df_complaint2020_h1_groupby = df_complaint2020_h1.groupby("Month").count()

In [ ]:
df_complaint2020_h1_groupby.head(1)

In [ ]:
col_list = ["CMPLNT_NUM"]
df_complaint2020_h1_groupby = df_complaint2020_h1_groupby[col_list]
df_complaint2020_h1_groupby['month'] = df_complaint2020_h1_groupby.index

In [ ]:
df_complaint2020_h1_groupby = df_complaint2020_h1_groupby.rename(columns={"CMPLNT_NUM": "TotalComplaints2020"})

In [ ]:
result = pd.merge(df_complaint2021_groupby, df_complaint2020_h1_groupby, on="month")

In [ ]:
result

In [ ]:
result["Chng2021_h1"] = (result['TotalComplaints2021'] - result['TotalComplaints2020'])/result.TotalComplaints2020*100

In [ ]:
result["Chng2021_h1"] = round(result["Chng2021_h1"],1)

In [ ]:
result["zero"] = 0

In [ ]:
import altair as alt
from vega_datasets import data

chart = alt.Chart(result).mark_circle(color="coral").encode(
    alt.X('month:T', title="Month"),
    alt.Y('Chng2021_h1:Q', title="Percent Change in Complaints"),
    alt.Size('TotalComplaints2021:Q',
        scale=alt.Scale(range=[10, 1200]),
        legend=alt.Legend(title='Number of Complaints in 2021'))).properties(
height=300, width=350, title="More police complaints have been filed this year than last year")

line = alt.Chart(result).mark_rule(color='gray').encode(
    alt.Y('zero:Q'),
    size=alt.SizeValue(3),
)

In [ ]:
chart + line

In [ ]:
df_2021Murder = df_complaint2021[df_complaint2021.OFNS_DESC == "MURDER & NON-NEGL. MANSLAUGHTER"]

In [ ]:
# IMPORTANT 
df_2021Murder.value_counts("VIC_RACE", normalize=True) * 100

In [ ]:
df_2020Murder = df_complaint2020[df_complaint2020.Crime == "MURDER & NON-NEGL. MANSLAUGHTER"]

In [ ]:
# IMPORTANT 
df_2020Murder.value_counts("VIC_RACE", normalize=True) * 100

In [ ]:
# IMPORTANT

import geopandas as gpd

ntaData = gpd.read_file\
('https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/ArcGIS/rest/services/NYC_Neighborhood_Tabulation_Areas/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson')

import altair as alt

ntaMap = alt.Chart(ntaData).mark_geoshape(
    fill='whitesmoke',
    stroke='gray',
    strokeWidth=0.5
).properties(
    width=500,
    height=600,
)

murderReports2020 = alt.Chart(df_2020Murder).mark_circle(
    filled=True, color='brown', size=12).encode(
    latitude='Latitude:Q',
    longitude='Longitude:Q',
    tooltip=[alt.Tooltip('PATROL_BORO', title='Borough'), alt.Tooltip('ADDR_PCT_CD', title='Precinct')]
    )

murderReports2021 = alt.Chart(df_2021Murder).mark_circle(
    filled=True, color='black', opacity=0.7, size=12).encode(
    latitude='Latitude:Q',
    longitude='Longitude:Q',
    tooltip=[alt.Tooltip('PATROL_BORO', title='Borough'), alt.Tooltip('ADDR_PCT_CD', title='Precinct')]
    )

alt.layer(ntaMap, murderReports2020, murderReports2021,\
          title="Murder reports in NYC"
)

In [ ]:
# df_complaint.Crime=df_complaint.Crime.replace({'ROBBERY':'Robbery',\
#                                                                        'FELONY ASSAULT':"Felony Assault"})

In [ ]:
# QUESTION: Overlap data by borough with mobility line graph 

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
# import re
# df_Robbery_FelonyAssault['Month']=df_Robbery_FelonyAssault.Date.apply(lambda txt: re.findall("^\d\d", txt))

In [ ]:
# df_Robbery_FelonyAssault['Month']=df_Robbery_FelonyAssault['Month'].astype(str)

In [ ]:
# df_Robbery_FelonyAssault.Month = df_Robbery_FelonyAssault.Month.str.replace("[","")
# df_Robbery_FelonyAssault.Month = df_Robbery_FelonyAssault.Month.str.replace("]","")
# df_Robbery_FelonyAssault.Month = df_Robbery_FelonyAssault.Month.str.replace("'","")

In [ ]:
# crimeCount = df_complaint.groupby("Date")['Crime'].count()
# df_crimeCount = pd.DataFrame([crimeCount])
# df_crimeCount = df_crimeCount.T
# df_crimeCount['Date'] = df_crimeCount.index

In [ ]:
# df_crimeCount['sevenMajorCrimes'] = "crimes"

In [ ]:
# df_crimeCount['%change']=df_crimeCount['Crime'].pct_change()*100

In [ ]:
# df_crimeCount['%change'] = df_crimeCount['%change'].fillna(0)

In [ ]:
# # IMPORTANT - %change has to be YoY
# alt.Chart(df_crimeCount).mark_line().encode(
#    x="Date:T",
#    y="Crime:Q",
#    color="sevenMajorCrimes"
# ).properties(
#    height=200, width=300
# ) 

# Step 6.1: Property crime (burglary, grand larceny, motor vehicle theft)
### nonviolent crimes dropped -3.3% YoY in Jan-July 2021 & and rose 1% in 2020
#### in top 10 precincts with biggest increase, three crimes rose across the board
#### GL was flat 2016 - 2019 and DROPPED in 2020; vehicle theft, burglary was flat 2016-2019 and SURGED in 2020

In [ ]:
NonViolent_Crimes = df_MF4[(df_MF4['CRIME'] == "GRAND LARCENY") |\
                           (df_MF4['CRIME'] == "GRAND LARCENY OF MOTOR VEHICLE") |\
                           (df_MF4['CRIME'] == "BURGLARY")]
NonViolent_Crimes = NonViolent_Crimes[['PCT', 'CRIME', '2021_YTD', '2020_YTD', '%_Chg','household_median_income']]

In [ ]:
NonViolent_Crimes.head()

In [ ]:
(NonViolent_Crimes['2021_YTD'].sum() - NonViolent_Crimes['2020_YTD'].sum()) / NonViolent_Crimes['2020_YTD'].sum() * 100

In [ ]:
NonViolent_Crimes[NonViolent_Crimes['%_Chg'] > -2.7].household_median_income.median()

In [ ]:
NonViolent_Crimes[NonViolent_Crimes['%_Chg'] < -2.7].household_median_income.median()

In [ ]:
NonViolent_Crimes.head(1)

In [ ]:
annual_NonViolentCrimes = df_MF2.loc[["GRAND LARCENY", "GRAND LARCENY OF MOTOR VEHICLE", "BURGLARY"]]

In [ ]:
(annual_NonViolentCrimes[2020.0].sum() - annual_NonViolentCrimes[2019.0].sum()) / annual_NonViolentCrimes[2019.0].sum() * 100

In [ ]:
NonViolentCrimes2021 = NonViolent_Crimes.groupby("PCT")['2021_YTD'].sum()
NonViolentCrimes2021 = NonViolentCrimes2021.to_frame()
NonViolentCrimes2021.reset_index(level=0, inplace=True)
NonViolentCrimes2021 = NonViolentCrimes2021.sort_values(by="PCT", ascending = True)

In [ ]:
NonViolentCrimes2020 = NonViolent_Crimes.groupby("PCT")['2020_YTD'].sum()
NonViolentCrimes2020 = NonViolentCrimes2020.to_frame()
NonViolentCrimes2020.reset_index(level=0, inplace=True)
NonViolentCrimes2020 = NonViolentCrimes2020.sort_values(by="PCT", ascending = True)

In [ ]:
NonViolentCrimes_Precinct = pd.merge(NonViolentCrimes2020, NonViolentCrimes2021, on='PCT', how='outer')
NonViolentCrimes_Precinct['%chng'] = (NonViolentCrimes_Precinct['2021_YTD'] - NonViolentCrimes_Precinct['2020_YTD']) / NonViolentCrimes_Precinct['2020_YTD'] * 100
NonViolentCrimes_Precinct['%chng'] = round(NonViolentCrimes_Precinct['%chng'], 1)

In [ ]:
# IMPORTANT
NonViolentCrimes_Precinct.sort_values(by="%chng", ascending=False).head(10)

In [ ]:
# spot checking
NonViolent_Crimes[NonViolent_Crimes.PCT == 114]

In [ ]:
# Grand Larceny
GL = NonViolent_Crimes[NonViolent_Crimes.CRIME == 'GRAND LARCENY']

In [ ]:
GL.to_csv("GrandLarceny2021.csv", index=False)

In [ ]:
# Burglary
Burglary = NonViolent_Crimes[NonViolent_Crimes.CRIME == 'BURGLARY']

In [ ]:
Burglary.to_csv("Burglary2021.csv", index=False)

In [ ]:
# vehicle theft
VehicleTheft = NonViolent_Crimes[NonViolent_Crimes.CRIME == 'GRAND LARCENY OF MOTOR VEHICLE']

In [ ]:
VehicleTheft = VehicleTheft[VehicleTheft['%_Chg']>0]

In [ ]:
VehicleTheft.to_csv("VehicleTheft2021.csv", index=False)

# Shootings 

In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_colwidth", 200)

In [ ]:
import requests
import json

In [ ]:
df_shootings2021 = pd.read_csv('NYPD_Shooting_Incident_Data__Year_To_Date_.csv')

In [ ]:
df_shootings2021.head(1)

In [ ]:
df_shootings2021['year'] = df_shootings2021.OCCUR_DATE

In [ ]:
df_shootings2021['year'] = df_shootings2021['year'].astype(str)

In [ ]:
import datetime as dt

In [ ]:
import re

In [ ]:
df_shootings2021['year'] = df_shootings2021['year'].apply(lambda txt: re.findall("\d\d\d\d", txt))

In [ ]:
df_shootings2021['year'] = df_shootings2021['year'].astype(str)

In [ ]:
df_shootings2021['year'] = df_shootings2021['year'].str.strip('[]').str.strip("''").astype(int)

In [ ]:
df_shootings2021.head(5)

In [ ]:
df_shootings2021 = df_shootings2021[df_shootings2021['year'] == 2021]

In [ ]:
df_shootings2021.shape

In [ ]:
import geopandas as gpd

ntaData = gpd.read_file\
('https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/ArcGIS/rest/services/NYC_Neighborhood_Tabulation_Areas/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson')

import altair as alt

ntaMap = alt.Chart(ntaData).mark_geoshape(
    fill='whitesmoke',
    stroke='gray',
    strokeWidth=0.5
).encode(tooltip=[alt.Tooltip('Precinct')])

ntaMap

In [ ]:
df_shootings2021.head(1)

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
# IMPORTANT
ntaMap = alt.Chart(ntaData).mark_geoshape(
    fill='whitesmoke',
    stroke='gray',
    strokeWidth=0.5
).properties(
    width=600,
    height=700
)

shootingMap2021 = alt.Chart(df_shootings2021).mark_circle(filled=True, color='orange', size=7, strokeWidth=0.2, stroke='black').encode(
    latitude='Latitude:Q',
    longitude='Longitude:Q',
    color='year:N',
    tooltip=[alt.Tooltip('OCCUR_DATE', title="Occurance date"), alt.Tooltip('PRECINCT', title="Precinct"),\
             alt.Tooltip('BORO',title="Borough"), alt.Tooltip('LOCATION_DESC', title="Location description")])\
.properties(
    title='Shootings in NYC, Jan.-June 2021',
    width=600,
    height=700
)

shootings2021 = alt.layer(ntaMap, shootingMap2021)

In [ ]:
shootings2021

## historic shooting data

In [ ]:
df_shootings_historic = pd.read_csv('NYPD_Shooting_Incident_Data__Historic_.csv')

In [ ]:
df_shootings_historic.shape

In [ ]:
# import geopandas as gpd

In [ ]:
# cityPropertiesGeo = gpd.read_file('https://data.cityofnewyork.us/resource/833y-fsy8.geojson')

In [ ]:
df_shootings_historic.head(2)

In [ ]:
df_shootings_historic['year'] = pd.to_datetime(df_shootings_historic.OCCUR_DATE)

In [ ]:
df_shootings_historic['year'] = df_shootings_historic['year'].dt.year

In [ ]:
df_shootings_historic['month'] = pd.to_datetime(df_shootings_historic.OCCUR_DATE)

In [ ]:
df_shootings_historic['month'] = df_shootings_historic['month'].dt.month

In [ ]:
df_shootings_historic.head(2)

In [ ]:
df_fiveYear_shootings =  df_shootings_historic[(df_shootings_historic.year == 2015) | (df_shootings_historic.year == 2016)|\
    (df_shootings_historic.year == 2017) |(df_shootings_historic.year == 2018)|\
                                       (df_shootings_historic.year == 2019) |(df_shootings_historic.year == 2020)]

In [ ]:
shootings2020 = df_fiveYear_shootings[df_fiveYear_shootings.year == 2020]

In [ ]:
shootings2020_h2 = shootings2020[(shootings2020.month == 1)| (shootings2020.month == 2) |(shootings2020.month == 3) |\
             (shootings2020.month == 4) | (shootings2020.month == 5) | (shootings2020.month == 6)]

In [ ]:
##DO NOT ERASE, KEEP THIS COLUMN!!!

## grouping shootings by precinct

# df_shootings2021_groupby = df_shootings2021.groupby("PRECINCT").count()

# col_list = ['INCIDENT_KEY']
# df_shootings2021_groupby = df_shootings2021_groupby[col_list]
# df_shootings2021_groupby = df_shootings2021_groupby.rename(columns={"INCIDENT_KEY": "Shootings_2021_h2"})

# shootings2020_h2_groupby = shootings2020_h2.groupby("PRECINCT").count()

# col_list = ['INCIDENT_KEY']
# shootings2020_h2_groupby = shootings2020_h2_groupby[col_list]
# shootings2020_h2_groupby = shootings2020_h2_groupby.rename(columns={"INCIDENT_KEY": "Shootings_2020_h2"})

# df_shootings2021_groupby.reset_index(inplace=True)
# shootings2020_h2_groupby.reset_index(inplace=True)

# dfinal = pd.merge(df_shootings2021_groupby, shootings2020_h2_groupby, on="PRECINCT", how="outer")

# dfinal = dfinal.fillna(0)

# dfinal = dfinal.rename(columns={"PRECINCT": "PCT"})

# dfinal['Chg2021'] = (dfinal.Shootings_2021_h2 - dfinal.Shootings_2020_h2)/dfinal.Shootings_2020_h2*100

# dfinal['Chg2021'] = round(dfinal['Chg2021'], 1)

# dfinal['CRIME'] = "Shooting"

# # col_list = ['PCT', 'household_median_income']
# # dfinal = dfinal[col_list]

# temp = df_d3[df_d3.CRIME.str.strip() == "Murder"]

# col_list = ['PCT', 'Chg2021', 'CRIME']
# dfinal  = dfinal[col_list]

# dfinal_new = pd.merge(dfinal, temp, on="PCT", how="outer")
# dfinal_new.CRIME_x = dfinal_new.CRIME_x.fillna("Shooting")
# dfinal_new.Chg2021_x = dfinal_new.Chg2021_x.fillna(0)

## dfinal_new.to_csv("dfinal_new.csv",index=False)

In [ ]:
dfinal_new2

In [ ]:
df_fiveYear_shootings.head(1)

In [ ]:
# IMPORTANT
ntaMap = alt.Chart(ntaData).mark_geoshape(
    fill='whitesmoke',
    stroke='gray',
    strokeWidth=0.5
).properties(
    width=600,
    height=700
)

fiveYear_shootingMap = alt.Chart(df_fiveYear_shootings).mark_circle(filled=True, opacity=0.7, size=6, strokeWidth=0.15, stroke='black').encode(
    latitude='Latitude:Q',
    longitude='Longitude:Q',
    color='year:N',
    tooltip=[alt.Tooltip('OCCUR_DATE', title="Occurance date"), alt.Tooltip('PRECINCT', title="Precinct"),\
             alt.Tooltip('BORO',title="Borough"), alt.Tooltip('LOCATION_DESC', title="Location description")])\
.properties(
    title='Shootings in NYC, 2015 - 2020',
    width=600,
    height=700
)

fiveYear_shooting  = alt.layer(ntaMap, fiveYear_shootingMap )

In [ ]:
fiveYear_shooting

In [ ]:
shootings2021 | fiveYear_shooting

In [ ]:
df_shootings1920 = df_shootings_historic[(df_shootings_historic.year == 2019) |(df_shootings_historic.year == 2020)]

In [ ]:
df_shootings2019 = df_shootings_historic[df_shootings_historic.year == 2019]

In [ ]:
df_shootings2019.shape

In [ ]:
df_shootings2020map = df_shootings_historic[df_shootings_historic.year == 2020]

In [ ]:
df_shootings2020map

In [ ]:
df_shootings2018 = df_shootings_historic[df_shootings_historic.year == 2018]

In [ ]:
df_shootings2018.shape

In [ ]:
df_shootings2019.shape

In [ ]:
1918/2

In [ ]:
(1942 - 959) / 959 * 100

In [ ]:
df_shootings1819 = df_shootings_historic[(df_shootings_historic.year == 2018) | (df_shootings_historic.year == 2019)]

In [ ]:
df_shootings1819_new = df_shootings1819.groupby("PRECINCT").count()
df_shootings1819_new

In [ ]:
df_shootings1819_new['Precinct'] = df_shootings1819_new.index

col_list = ['Precinct','INCIDENT_KEY']
df_shootings1819_new = df_shootings1819_new[col_list]

In [ ]:
df_shootings1819_new['Avg_Shootings_2018_2019'] = df_shootings1819_new.INCIDENT_KEY/2

In [ ]:
df_shootings1819_new

In [ ]:
df_shootings2020 = df_shootings_historic[df_shootings_historic.year == 2020]

In [ ]:
df_shootings2020.shape

In [ ]:
col_list = ['INCIDENT_KEY','PRECINCT']
df_shootings2020 = df_shootings2020[col_list]

In [ ]:
df_shootings2020_new = df_shootings2020.groupby("PRECINCT").count()
df_shootings2020_new['Precinct'] = df_shootings2020_new.index

In [ ]:
df_shootings2020_new.head()

In [ ]:
df_shootings1819_new[df_shootings1819_new.Precinct == 104]

In [ ]:
df_shootings_final_new = df_shootings1819_new.merge(df_shootings2020_new, left_on='Precinct', right_on='Precinct', how='outer')

In [ ]:
df_shootings_final_new.head(1)

In [ ]:
df_shootings_final_new = df_shootings_final_new.rename(columns={"INCIDENT_KEY_y":"Shootings2020"})

In [ ]:
df_shootings_final_new

In [ ]:
df_shootings_final_new = df_shootings_final_new.drop(columns=['INCIDENT_KEY_x'])

In [ ]:
df_shootings_final_new

In [ ]:
df_shootings_final_new['%_Change_Shootings_2020'] = (df_shootings_final_new['Shootings2020']- df_shootings_final_new['Avg_Shootings_2018_2019'])/ df_shootings_final_new['Avg_Shootings_2018_2019'] * 100

In [ ]:
df_shootings_final_new['%_Change_Shootings_2020'] = round(df_shootings_final_new['%_Change_Shootings_2020'],1)

In [ ]:
df_shootings_final_new.sort_values(['%_Change_Shootings_2020'], ascending = True).head(20)

In [ ]:
df_shootings_final_new.to_csv("ShootingsPercentChange.csv", index=False)

## homicide data
#### homicides have been on an uptick since 2017; soared in 2020; and rose again Jan-July 2021
#### 64% of homicides in 2020 used guns
#### 51% of homicides in 2019 used guns
#### 54% of homicides in 2018 used guns

In [ ]:
# https://www1.nyc.gov/site/nypd/stats/reports-analysis/homicide.page
df_homicide2020 = pd.read_excel("supplementary-homicide-report-cy2020 (1).xlsx")

In [ ]:
df_homicide2020.head()

In [ ]:
new_header = df_homicide2020.iloc[3] #grab the first row for the header
df_homicide2020 = df_homicide2020[4:] #take the data less the header row
df_homicide2020.columns = new_header #set the header row as the df header

In [ ]:
df_homicide2020.head(2)

In [ ]:
df_homicide2020.PRECINCT = df_homicide2020.PRECINCT.astype(int)

In [ ]:
df_homicide2020.to_csv("homicides2020.csv", index=False)

In [ ]:
df_homicide2020.value_counts("WEAPON USE")

In [ ]:
29500/464

In [ ]:
df_homicide2020_new = df_homicide2020.value_counts("PRECINCT")

In [ ]:
df_homicide2020_new = df_homicide2020_new.to_frame()

In [ ]:
df_homicide2020_new

In [ ]:
df_homicide2020_new.reset_index(level=0, inplace=True)

In [ ]:
df_homicide2020_new = df_homicide2020_new.rename(columns={0: "NumOfHomicides"})

In [ ]:
df_homicide2020_new['percentage'] = df_homicide2020_new.NumOfHomicides / 69 * 100
df_homicide2020_new['percentage'] = round(df_homicide2020_new['percentage'],1)

In [ ]:
df_homicide2020_new.PRECINCT = df_homicide2020_new.PRECINCT.astype(int)

In [ ]:
df_homicide2020_new

In [ ]:
df_homicide2020_new.to_csv("homicides2020.csv", index=False)

In [ ]:
df_homicide2019 = pd.read_excel("supplementary-homicide-report-cy2019.xlsx")

In [ ]:
new_header = df_homicide2019.iloc[3] #grab the first row for the header
df_homicide2019 = df_homicide2019[4:] #take the data less the header row
df_homicide2019.columns = new_header #set the header row as the df header

In [ ]:
len(df_homicide2019)

In [ ]:
df_homicide2019.value_counts("WEAPON USE")

In [ ]:
16300/319

In [ ]:
df_homicide2019_new = df_homicide2019.value_counts("PRECINCT")
df_homicide2019_new = df_homicide2019_new.to_frame()
df_homicide2019_new.reset_index(level=0, inplace=True)
df_homicide2019_new = df_homicide2019_new.rename(columns={0: "NumOfHomicides"})

In [ ]:
df_homicide2019_new['percentage'] = df_homicide2019_new.NumOfHomicides / 69 * 100
df_homicide2019_new['percentage'] = round(df_homicide2019_new['percentage'],1)

In [ ]:
df_homicide2019_new.PRECINCT = df_homicide2019_new.PRECINCT.astype(int)

In [ ]:
df_homicide2019_new.to_csv("homicides2019.csv", index=False)

In [ ]:
df_homicide2018 = pd.read_excel("supplementary-homicide-report-cy2018.xlsx")

In [ ]:
new_header = df_homicide2018.iloc[3] #grab the first row for the header
df_homicide2018 = df_homicide2018[4:] #take the data less the header row
df_homicide2018.columns = new_header #set the header row as the df header

In [ ]:
df_homicide2018.value_counts("WEAPON USE")

In [ ]:
len(df_homicide2018)

In [ ]:
16100/295

In [ ]:
df_homicide2018_new = df_homicide2018.value_counts("PRECINCT")
df_homicide2018_new = df_homicide2018_new.to_frame()
df_homicide2018_new.reset_index(level=0, inplace=True)
df_homicide2018_new = df_homicide2018_new.rename(columns={0: "NumOfHomicides"})

In [ ]:
df_homicide2018_new['percentage'] = df_homicide2018_new.NumOfHomicides / 69 * 100
df_homicide2018_new['percentage'] = round(df_homicide2018_new['percentage'],1)

In [ ]:
df_homicide2018_new.PRECINCT = df_homicide2018_new.PRECINCT.astype(int)

In [ ]:
df_homicide2018_new.to_csv("homicides2018.csv", index=False)

# assign income tiers
### lower: - 54103.5
### lower-middle: 54103.5 - 68920.0
### upper-midde: 68920.0 - 91735.0
### upper: 91735.0+

In [ ]:
os.chdir("MedianIncomeData")

In [ ]:
df_MI = pd.read_csv("df_MI.csv")

In [ ]:
df_MI['Median household income2019'].isna().sum()

In [ ]:
# IMPORTANT
a = np.array([df_MI['Median household income2019']])
fifty = np.percentile(a, 50) # return 50th percentile, e.g median.

In [ ]:
fifty

In [ ]:
twentyfive = np.percentile(a, 25) # return 25th percentile, e.g low

In [ ]:
twentyfive

In [ ]:
seventyfive = np.percentile(a, 75) # return 75th percentile, e.g up

In [ ]:
seventyfive

## scatter plot map % change in crimes & income tiers

In [ ]:
df_sevenMajorIncome = df_MF4[df_MF4.CRIME == "TOTAL SEVEN MAJOR FELONY OFFENSES"]

In [ ]:
df_sevenMajorIncome.head(2)

In [ ]:
df_sevenMajorIncome.columns = df_sevenMajorIncome.columns.astype(str)

In [ ]:
# lower income: - 54103.5

In [ ]:
lower = df_sevenMajorIncome[df_sevenMajorIncome.household_median_income < 54103.5]

In [ ]:
lower['Income_Tier'] = "Lower"

In [ ]:
len(lower)

In [ ]:
(lower['2021_YTD'].sum() - lower['2020_YTD'].sum()) / lower['2019'].sum() * 100

In [ ]:
# lower-middle: 54103.5 - 68920.0

In [ ]:
lowerMiddle = df_sevenMajorIncome[(df_sevenMajorIncome.household_median_income > 54103.6) &\
                                 (df_sevenMajorIncome.household_median_income < 68920.0)]

In [ ]:
lowerMiddle['Income_Tier'] = "Lower Middle"

In [ ]:
len(lowerMiddle)

In [ ]:
(lowerMiddle['2020'].sum() - lowerMiddle['2019'].sum()) / lowerMiddle['2019'].sum() * 100

In [ ]:
(lower['2021_YTD'].sum() + lowerMiddle['2021_YTD'].sum()) - (lower['2020_YTD'].sum()+lowerMiddle['2020_YTD'].sum())

In [ ]:
lower['2020_YTD'].sum()+lowerMiddle['2020_YTD'].sum()

In [ ]:
143/29615*100

In [ ]:
# upper-midde: 68920.0 - 91735.0

In [ ]:
upperMiddle = df_sevenMajorIncome[(df_sevenMajorIncome.household_median_income > 68920.1) &\
                                 (df_sevenMajorIncome.household_median_income < 91735.0)]

In [ ]:
upperMiddle['Income_Tier'] = "Upper Middle"

In [ ]:
len(upperMiddle)

In [ ]:
(upperMiddle['2020'].sum() - upperMiddle['2019'].sum()) / upperMiddle['2019'].sum() * 100

In [ ]:
# upper: 91735.0+

In [ ]:
upper = df_sevenMajorIncome[df_sevenMajorIncome.household_median_income > 91735.1]

In [ ]:
upper['Income_Tier'] = "Upper"

In [ ]:
len(upper)

In [ ]:
(upper['2020'].sum() - upper['2019'].sum()) / upper['2019'].sum() * 100

In [ ]:
(upperMiddle['2021_YTD'].sum() + upper['2021_YTD'].sum()) - (upperMiddle['2020_YTD'].sum() +upper['2020_YTD'].sum())

In [ ]:
upperMiddle['2020_YTD'].sum() +upper['2020_YTD'].sum()

In [ ]:
-504 / 16816 * 100

In [ ]:
frames = [lower, lowerMiddle, upperMiddle, upper]

result = pd.concat(frames)

In [ ]:
result.head(1)

In [ ]:
result=result.rename(columns={"%_chg_2019-2020": "Percent_change_between_2019-2020", "%_Chg": "Percent_change_between_2020-2021",\
                             "household_median_income":"Household_Median_Income"
                             })

In [ ]:
result = result[result.PCT!=22]

In [ ]:
import altair as alt
from vega_datasets import data

incomeTier2020 = alt.Chart(result).mark_circle(size=60).encode(
    alt.X('Household_Median_Income:Q'),
    alt.Y('Percent_change_between_2019-2020:Q',
        scale=alt.Scale(domain=(-40, 40))),
    color='Income_Tier',
    tooltip=['PCT:N', '%_chg_2019-2020:N']
).properties(
    width=450,
    height=300,
    title = 'Change in major crimes by income tier in NYC in 2020'
)

final_plot2020 = incomeTier2020 + incomeTier2020.transform_regression('Household_Median_Income','Percent_change_between_2019-2020').mark_line()

final_plot2020

In [ ]:
# final.save('chart.html')

In [ ]:
## plotting for 2021

In [ ]:
incomeTier2021 = alt.Chart(result).mark_circle(size=60).encode(
    alt.X('Household_Median_Income:Q'),
    alt.Y('Percent_change_between_2020-2021:Q',
        scale=alt.Scale(domain=(-40, 40))),
    color='Income_Tier',
    tooltip=['PCT:N', 'Percent_change_between_2020-2021:N']
).properties(
    width=450,
    height=300,
    title = 'Change in major crimes by income tier in NYC, Jan.-July 2021'
) 

# making the regression line using transform_regression
# function and add with the scatter plot
final_plot2021 = incomeTier2021 + incomeTier2021.transform_regression('Household_Median_Income','Percent_change_between_2020-2021').mark_line()

In [ ]:
final_plot2021

In [ ]:
final_plot2020|final_plot2021